In [1]:
import numpy as np
import pandas as pd
import re
import requests
import json

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_colwidth', 1000000)

In [4]:
pd.reset_option('display.max_colwidth')

In [2]:
df = pd.read_csv('/home/roland/Projects/JP_citation_classification/feature_scraping/data/enriched_data/6_diff_in_publication_date.csv')

In [5]:
papers = pd.read_csv('data/causeminer_data/papers_with_core_id.csv')

In [2]:
sent = pd.read_csv('data/causeminer_data/sentences_with_core_id.csv')

In [6]:
sent_2 = pd.read_csv('data/causeminer_data_2/sentences.csv')

In [4]:
ref = pd.read_csv('data/causeminer_data/references_with_core_id.csv')

In [13]:
ref2 = pd.read_csv('data/causeminer_data_2/references_raw.csv')

In [14]:
len(ref2)

14133

In [15]:
len(ref)

14133

In [6]:
cite = pd.read_csv('data/causeminer_data/citations_with_pos.csv')

# matching causeminer data

## matching article_id / core_id

In [24]:
core_id_to_filename = {
81153632: 'HohensinnerS_2013_Two steps back, one step forward: reconstructing the dynamic Danube riverscape under human influence in Vienna_Water History',
 50694047: 'JensenO_2013_Reproducibility of tender point examination in chronic low back pain patients as measured by intrarater and inter-rater reliability and agreement: a validation study_BMJ Open',
 5225915: 'RobinsonE_2011_A Simple Threshold Rule Is Sufficient to Explain Sophisticated Collective Decision-Making_PLoS ONE',
 82875399: 'HeatherN_2014_Interpreting Null Findings from Trials of Alcohol Brief Interventions_Frontiers in Psychiatry',
 82218905: 'Funck-BrentanoC_2013_Effects of rabeprazole on the antiplatelet effects and pharmacokinetics of clopidogrel in healthy volunteers_Archives of Cardiovascular Diseases',
 30272741: 'BakerD_2015_Brain networks underlying bistable perception_NeuroImage',
 80689400: 'ValleL_2015_Official Statistics Data Integration for Enhanced Information Quality_Quality and Reliability Engineering International',
 56377383: 'KennettM_2013_Out-of-Equilibrium Dynamics of the Bose-Hubbard Model_ISRN Condensed Matter Physics',
 81848562: 'HussainS_2016_Streams as Entanglement of Nature and Culture: European Upper Paleolithic River Systems and Their Role as Features of Spatial Organization_Journal of Archaeological Method and Theory',
 397819: 'HegartyP_2010_A stone in the soup. Changes in sexual prejudice and essentialist beliefs among british students in a class on lgbt psychology_Psychology and Sexuality',
 81158461: 'RodejohannW_2008_The see-saw mechanism: Neutrino mixing, leptogenesis and lepton flavour violation_Pramana',
 50694980: 'ThorlundJ_2013_Knee Arthroscopy Cohort Southern Denmark (KACS): protocol for a prospective cohort study_BMJ Open',
 36200578: 'DiushiK_2015_Evolutionary History of the Live-Bearing Endemic Allotoca diazi Species Complex (Actinopterygii, Goodeinae): Evidence of Founder Effect Events in the Mexican Pre-Hispanic Period_PLOS ONE',
 82865416: 'PartonenT_2012_Hypothesis: Cryptochromes and Brown Fat are Essential for Adaptation and Affect Mood and Mood-Related Behaviors_Frontiers in Neurology',
 76981266: 'HaddadM_2013_Promoting mental health in men_Nursing Standard',
 81075756: 'SwengelA_2010_The butterfly fauna of Wisconsin bogs: lessons for conservation_nan',
 36209468: 'VallisG_2014_Response of Atmospheric Structure to Global Warming_nan',
 20349879: 'MitchardE_2014_Markedly divergent estimates of Amazon forest carbon density from ground plots and satellites_Global Ecology and Biogeography',
 81542265: 'OnofrjM_2013_Cohort study of prevalence and phenomenology of tremor in dementia with Lewy bodies_Journal of Neurology',
 5255546: 'JankaH_2012_Explosion Mechanisms of Core-Collapse Supernovae_Annual Review of Nuclear and Particle Science',
 161122997: 'StainN_2014_Gender comparisons in non-acute cardiac symptom recognition and subsequent help-seeking decisions: a mixed methods study protocol_BMJ Open',
 211327317: "JokonyaO_2019_Towards a Framework for Decision Making Regarding IT Adoption_Proceedings of the South African Institute for Computer Scientists and Information Technologists Conference on - SAICSIT '12 the South African Institute for Computer Scientists and Information Technologists Conference on - SAICSIT '12",
 35467497: 'WiltshireD_2011_What is dust?-Physical foundations of the averaging problem in cosmology_Classical and Quantum Gravity',
 30893585: 'GeysB_2012_Association Membership and Generalized Trust: Are Connections between Associations Losing their Value?_Journal of Civil Society',
 2099218: 'YadavJ_2010_Fractal Dimension as a measure of the scale of Homogeneity_Monthly Notices of the Royal Astronomical Society',
 30710187: 'StrubbeD_2015_Invasion success of a global avian invader is explained by within-taxon niche structure and association with humans in the native range_Diversity and Distributions',
 2073440: 'FairhurstS_2009_Triangulation of gravitational wave sources with a network of detectors_New Journal of Physics',
 71826501: 'OlmedillasH_2011_Bone Related Health Status in Adolescent Cyclists_PLoS ONE',
 82878092: 'ZilberterT_2011_Carbohydrate-Biased Control of Energy Metabolism: The Darker Side of the Selfish Brain_Frontiers in Neuroenergetics',
 43173419: 'RommN_2013_Active facilitation of focus groups: exploring the implementation of inclusive education with research participants_nan',
 30903216: 'QuintanaD_2014_Considerations in the assessment of heart rate variability in biobehavioral research_Frontiers in Psychology',
 33740162: 'SpillingK_2015_Interaction Effects of Light, Temperature and Nutrient Limitations (N, P and Si) on Growth, Stoichiometry and Photosynthetic Parameters of the Cold-Water Diatom Chaetoceros wighamii_PLOS ONE',
 30269551: 'BensonP_2015_What is the value of orthodontic treatment?_British Dental Journal',
 81065436: 'VuurenD_2014_A new scenario framework for Climate Change Research: scenario matrix architecture_Climatic Change',
 30791123: 'ChowdhuryR_2015_Breastfeeding and maternal health outcomes: a systematic review and meta-analysis_Acta Paediatrica',
 187936510: 'FilippinA_2016_A Reconsideration of Gender Differences in Risk Attitudes A Reconsideration of Gender Differences in Risk Attitudes_SSRN Electronic Journal',
 10910685: 'AssociateH_2012_A genealogy of calculations at an early Queensland sugar mill_Accounting History',
 81074389: 'ElmqvistT_2013_History of Urbanization and the Missing Ecology_Urbanization, Biodiversity and Ecosystem Services: Challenges and Opportunities',
 13505159: 'RathjenM_2012_From the weak to the strong existence property_Annals of Pure and Applied Logic',
 77127873: 'KarlssonI_2014_Fungicide Effects on Fungal Community Composition in the Wheat Phyllosphere_PLoS ONE',
 186926157: 'DamayantiI_2017_FROM STORYTELLING TO STORY WRITING: THE IMPLEMENTATION OF READING TO LEARN (R2L) PEDAGOGY TO TEACH ENGLISH AS A FOREIGN LANGUAGE IN INDONESIA_Indonesian Journal of Applied Linguistics',
 55621548: 'CortezP_2012_Evolutionary Support Vector Machines for Time Series Forecasting_Artificial Neural Networks and Machine Learning – ICANN 2012',
 30708902: 'DakanalisA_2014_Testing the cognitive-behavioural maintenance models across DSM-5 bulimic-type eating disorder diagnostic groups: a multi-centre study_European Archives of Psychiatry and Clinical Neuroscience',
 81896289: 'HellwigM_2019_Systemic Risk in the Financial Sector: An Analysis of the Subprime-Mortgage Financial Crisis_De Economist',
 81880580: 'FanselowG_2011_Left peripheral focus: mismatches between syntax and information structure_Natural Language & Linguistic Theory',
 82105846: 'WachholzP_2014_Evidence on the role of prebiotics, probiotics, and synbiotics in gut health and disease prevention in the elderly_Journal of Clinical Gerontology and Geriatrics',
 76526352: 'LupettiM_2015_From a Robotic Vacuum Cleaner to a Robot Companion: Acceptance and Engagement in Domestic Environments_Proceedings of the Tenth Annual ACM/IEEE International Conference on Human-Robot Interaction Extended Abstracts the Tenth Annual ACM/IEEE International Conference on Human-Robot Interaction Extended Abstracts',
 55621512: 'CarvalhoP_2012_Memory Encoding of Stimulus Features in Human Perceptual Learning_Journal of Cognitive Psychology',
 20347437: 'CableN_2014_Critical role of smoking and household dampness during childhood for adult phlegm and cough: a research example from a prospective cohort study in Great Britain_BMJ Open',
 9636910: "ShepperdM_2012_The scientific basis for prediction research_Proceedings of the 8th International Conference on Predictive Models in Software Engineering - PROMISE '12 the 8th International Conference on Predictive Models in Software Engineering - PROMISE '12",
 76995576: 'HandD_2010_Evaluating diagnostic tests: the area under the ROC curve and the balance of errors_Statistics in Medicine',
 30269424: 'SkarmeasD_2014_Examining the role of CSR skepticism using fuzzy-set qualitative comparative analysis_Journal of Business Research',
 81153866: 'BelliniP_2014_Optimization of information retrieval for cross media contents in a best practice network_International Journal of Multimedia Information Retrieval',
 81822955: 'MatechouE_2016_Bayesian analysis of Jolly-Seber type models Incorporating heterogeneity in arrival and departure_Environmental and Ecological Statistics',
 82899510: 'FrankishK_1998_A matter of opinion_Philosophical Psychology',
 186893859: 'LangabeerS_2017_Case Report Molecular Profiling: A Case of ZBTB16-RARA Acute Promyelocytic Leukemia_Case Reports in Hematology',
 186893920: 'VolicerL_2014_Outcome Measures for Dementia in the Advanced Stage and at the End of Life_Advances in Geriatrics',
 186893992: 'LamR_2012_Which Depressive Symptoms and Medication Side Effects Are Perceived by Patients as Interfering Most with Occupational Functioning?_Depression Research and Treatment',
 186894183: 'MurrayN_2013_Clinical Study Circulating Prostate Cells Found in Men with Benign Prostate Disease Are P504S Negative: Clinical Implications_Journal of Oncology',
 186895161: 'TouliasT_2015_Generalized Information for the -Order Normal Distribution_Journal of Probability and Statistics',
 186895573: 'LowC_2017_5G Fixed Beam Switching on Microstrip Patch Antenna_International Journal of Electrical and Computer Engineering (IJECE)',
 186914012: 'BassilY_2011_An Image Steganography Scheme using Randomized Algorithm and Context-Free Grammar_Journal of Advanced Computer Science & Technology',
 186916832: 'NorgaardM_2016_Creating Under Pressure: Effects of Divided Attention on the Improvised Output of Skilled Jazz Pianists_Music Perception',
 186934554: 'WilcoxR_2015_ANCOVA: A GLOBAL TEST BASED ON A ROBUST MEASURE OF LOCATION OR QUANTILES WHEN THERE IS CURVATURE_nan',
 186951833: 'MorazaM_2006_TWO NEW TAXA OF PARHOLASPIDAE EVANS, 1956 (ACARI, MESOSTIGMATA) FROM TENERIFE (CANARY ISLANDS) _nan',
 186960882: 'NisbetE_2012_The regulation of the air: a hypothesis_Solid Earth',
 186961244: 'BerkovicS_2018_Wind regimes and their relation to synoptic variables using self-organizing maps_Advances in Science and Research',
 186961274: 'BerggrenM_2018_Quality transformation of dissolved organic carbon during water transit through lakes: contrasting controls by photochemical and biological processes_Biogeosciences',
 186959531: 'AlexanderD_2013_Resilience and disaster risk reduction: an etymological journey_Natural Hazards and Earth System Sciences',
 186964628: 'Brady-AmoonP_2017_Psychology, Counseling Psychology, and Professional Counseling: Shared Roots, Challenges, and Opportunities_The European Journal of Counselling Psychology',
 186969301: 'PinfieldS_2014_Open-access repositories worldwide, 2005-2012: Past growth, current characteristics and future possibilities_Journal of the Association for Information Science and Technology',
 186973270: 'AbundoM_2011_Optimal Admission Control for a QoS-aware Service-oriented System_nan',
 186976602: 'ForbesA_2013_Genetically differentiated races and speciation-with- gene-flow in the sunflower maggot, Strauzia longipennis_Evolutionary Ecology',
 187052037: 'FontehA_2006_Identification of disease markers in human cerebrospinal fluid using lipidomic and proteomic methods_Disease Markers',
 187052580: 'AlvesG_2016_A Case Study of MasterMind Chess: Comparing Mouse/Keyboard Interaction with Kinect-Based Gestural Interface_Advances in Human-Computer Interaction',
 187053012: 'IjazH_2017_Finite Element Analysis of Bend Test of Sandwich Structures Using Strain Energy Based Homogenization Method_Advances in Materials Science and Engineering',
 187053534: 'BaaP_2001_Application of Pfortran and Co-Array Fortran in the parallelization of the GROMOS96 molecular dynamics module_Scientific Programming',
 187053631: 'KalenderL_2016_REE Geochemistry of Euphrates River, Turkey_Journal of Chemistry',
 187054197: 'OksarM_2016_Case Report Is It Possible to Maintain Consciousness and Spontaneous Ventilation with Chest Compression in the Early Phase of Cardiac Arrest?_Case Reports in Anesthesiology',
 187054964: 'BaroneR_2014_Clinical Study Peritoneal Dialysis as a First versus Second Option after Previous Haemodialysis: A Very Long-Term Assessment_International Journal of Nephrology',
 189217163: "DrottJ_2016_The trajectory of neurotoxic side effects' impact on daily life: a qualitative study _Supportive Care in Cancer",
 158977742: 'ThanapalasingamT_2018_Ontology-Based Recommendation of Editorial Products_Lecture Notes in Computer Science',
 211273841: 'MyersC_2018_Co-designing Cards on Social Issues for Creating Educational Games_nan',
 159766439: 'PrideD_2018_Peer review and citation data in predicting university rankings, a large-scale analysis_Digital Libraries for Open Knowledge',
 42702124: 'IhliM_2016_Social network and content analysis of the North American Carbon Program as a scientific community of practice_Social Networks',
 81765890: 'GolcherH_2015_Neoadjuvant chemoradiation therapy with gemcitabine/cisplatin and surgery versus immediate surgery in resectable pancreatic cancer Results of the first prospective randomized phase II trial_Strahlentherapie und Onkologie',
 82864389: 'PapoD_2013_Time scales in cognitive neuroscience_Frontiers in Physiology',
 52119449: 'HeggdalK_2015_Patients Experience of the Outcomes of Engaging in a Broadly Applicable Health Promotion Intervention for Individuals Facing Chronic Illness_Health',
 30932364: 'NahrgangJ_2014_Gender Specific Reproductive Strategies of an Arctic Key Species (Boreogadus saida) and Implications of Climate Change_PLoS ONE',
 41241868: 'SmithA_2015_The Views and Experiences of Smokers Who Quit Smoking Unassisted. A Systematic Review of the Qualitative Evidence_PLOS ONE',
 81719097: 'PrideD_2017_Incidental or influential? Challenges in automatically detecting citation importance using publication full texts_Research and Advanced Technology for Digital Libraries',
 39414524: 'PencavelJ_2014_The Productivity of Working Hours_The Economic Journal',
 81091491: 'KingwellE_2015_High incidence and increasing prevalence of multiple sclerosis in British Columbia, Canada: findings from over two decades (1991-2010)_Journal of Neurology',
 11466311: 'NiculescuA_2008_Affordances in conversational interactions with multimodal QA systems_Lecture Notes in Computer Science',
 81280802: 'OttoF_2015_Attribution analysis of high precipitation events in summer in England and Wales over the last decade_Climatic Change',
 41236113: 'LuptonD_2012_M-HEALTH AND HEALTH PROMOTION: THE DIGITAL CYBORG AND SURVEILLANCE SOCIETY_nan',
 146503003: 'DumayN_2016_Sleep not just protects memories against forgetting, it also makes them more accessible_Cortex',
 18509190: 'HomburgV_2013_Diffusion of personalised services among Dutch municipalities: evolving channels of persuasion Paper submitted to Local Government Studies_nan',
 206252006: 'DietzS_2012_The Treatment of Risk and Uncertainty in the US Social Cost of Carbon for Regulatory Impact Analysis_SSRN Electronic Journal',
 81867052: 'DanchinA_2009_Information of the chassis and information of the program in synthetic cells_Systems and Synthetic Biology',
 82854584: 'CollertonD_2013_Psychotherapy and brain plasticity_Frontiers in Psychology',
 55853555: 'VanoutriveT_2012_WHAT DETERMINES CARPOOLING TO WORKPLACES IN BELGIUM: LOCATION, ORGANISATION, OR PROMOTION?_Journal of Transport Geography',
 30932485: 'BraekkanE_2014_Disentangling supply and demand shifts: the impacts on world salmon prices_Applied Economics',
 78076813: 'ZimmererV_2015_A case of "order insensitivity"? Natural and artificial language processing in a man with primary progressive aphasia_nan',
 141222918: 'NardiniM_2015_Integration of audio-visual information for spatial decisions in children and adults_Developmental Science',
 6503095: 'AkarcaA_2016_Turkish Voter Response to Government Incompetence and Corruption Related to the 1999 Earthquakes_SSRN Electronic Journal',
 33121888: 'RebullL_2015_A CATALOG OF POINT SOURCES TOWARD NGC 1333_The Astronomical Journal',
 29413232: 'MacdonaldA_2015_Routine Mental Health Outcome Measurement in the UK Running title: RCOM in the UK_International Review of Psychiatry',
 79496059: 'FreestoneI_2015_Glass Production at an Early Islamic Workshop in Tel Aviv_Journal of Archaeological Science',
 82260752: 'ChoC_2009_On the obstructed Lagrangian Floer theory_Advances in Mathematics',
 82846198: 'CumminsF_2014_Voice, (inter-)subjectivity, and real time recurrent interaction_Frontiers in Psychology',
 67538: 'BlowerG_2011_On the tau function associated with the generalized unitary ensemble_Journal of Statistical Physics',
 82403724: 'KleidonA_2010_Life, hierarchy, and the thermodynamic machinery of planet Earth_Physics of Life Reviews',
 82862405: 'HorstJ_2015_Category learning in a dynamic world_Frontiers in Psychology',
 55887336: 'SpruytA_2014_Feature-specific attention allocation modulates the generalization of recently acquired likes and dislikes_Experimental Psychology',
 82173079: "SingerF_2011_The Etiology of Paget's Disease of Bone: Viral and Genetic Interactions_Cell Metabolism",
 33604192: 'ChristenM_2014_Ethical focal points in the international practice of deep brain stimulation Ethical Focal Points in the International Practice of Deep Brain Stimulation_AJOB Neuroscience',
 82861940: 'MintonA_2014_The effect of time-dependent macromolecular crowding on the kinetics of protein aggregation: a simple model for the onset of age-related neurodegenerative disease_Frontiers in Physics',
 42611019: 'OlivolaC_2014_FACES OF LEADERSHIP The Many (Distinctive) Faces of Leadership: Inferring Leadership Domain from Facial Appearance_The Leadership Quarterly',
 79497095: 'MavranezouliI_2015_The Cost Effectiveness of Psychological and Pharmacological Interventions for Social Anxiety Disorder: A Model-Based Economic Analysis_PLOS ONE',
 82862948: 'WangH_2013_Reversing autism by targeting downstream mTOR signaling A commentary on Autism-related deficits via dysregulated eIF4E-dependent translational control_Frontiers in Cellular Neuroscience',
 36045385: 'HurkensS_2014_Mobile Termination, Network Externalities, and Consumer Expectations_nan',
 52129671: 'AadlandE_2015_Reliability of Objectively Measured Sedentary Time and Physical Activity in Adults_PLOS ONE',
 74390377: 'DubeyR_2015_Green Supply Chain Management Enablers: Mixed Methods Research_nan',
 81857737: 'PickersgillM_2013_From Implications to Dimensions: Science, Medicine and Ethics in Society_Health Care Analysis',
 24935271: 'MasseyP_2013_Massive Stars in the Galaxies of the Local Group_New Astronomy Reviews',
 81846122: 'GavazovK_2010_Isotopic analysis of cyanobacterial nitrogen fixation associated with subarctic lichen and bryophyte species_Plant and Soil',
 81768818: 'Diaz-UriarteR_2015_Identifying restrictions in the order of accumulation of mutations during tumor progression: effects of passengers, evolutionary models, and sampling_BMC Bioinformatics',
 30619848: 'MayJ_2011_Points in Mental Space: an Interdisciplinary Study of Imagery in Movement Creation_Dance Research',
 11277201: "WrigleyS_2010_Methodology and Campaign Design for the Evaluation of Semantic Search Tools_Proceedings of the 3rd International Semantic Search Workshop on - SEMSEARCH '10 the 3rd International Semantic Search Workshop on - SEMSEARCH '10",
 81799992: 'TorkzadM_2015_Indications and selection of MR enterography vs. MR enteroclysis with emphasis on patients who need small bowel MRI and general anaesthesia: results of a survey_Insights into Imaging',
 1352001: 'NakariakovV_2009_The possible role of vortex shedding in the excitation of kink-mode oscillations in the solar corona_Astronomy & Astrophysics',
 2136426: 'FairlieD_2010_Necessary conditions for Ternary Algebras_Journal of Physics A: Mathematical and Theoretical',
 9551787: 'ThorntonC_2012_Renewing the Link between Cognitive Archeology and Cognitive Science_Journal of Archaeological Science',
 46954401: 'HerrendorfB_2015_Why is Measured Productivity so Low in Agriculture?_nan',
 30273531: 'MossT_2015_Identification of the underlying factor structure of the Derriford Appearance Scale 24_PeerJ',
 2191277: 'ChernodubM_2011_Can nothing be a superconductor and a superfluid?_Proceedings of The many faces of QCD — PoS(FacesQCD) The many faces of QCD — PoS(FacesQCD)',
 19783337: 'CotterC_2011_Waltzing peakons and compacton pairs in a cross-coupled Camassa-Holm equation_Journal of Physics A: Mathematical and Theoretical',
 82874991: 'SansebastianoG_2013_Defining new SNARE functions: the i-SNARE_Frontiers in Plant Science',
 30319389: 'HibbardP_2015_Uncomfortable images produce non-sparse responses in a model of primary visual cortex_Royal Society Open Science',
 2185862: 'TakedaY_2011_Beryllium Abundances of Solar-Analog Stars _Publications of the Astronomical Society of Japan',
 82888546: 'SolomonovaE_2014_Methodological considerations for the neurophenomenology of dreaming: commentary on Windt\'s "Reporting dream experience"_Frontiers in Human Neuroscience',
 82852800: 'KlingerE_2013_Goal Commitments and the content of thoughts and dreams: basic principles_Frontiers in Psychology',
 33121562: 'SperhakeU_2014_The numerical relativity breakthrough for binary black holes_Classical and Quantum Gravity',
 55252268: 'KrieglerE_2014_The role of technology for achieving climate policy objectives: overview of the EMF 27 study on global technology and climate policy strategies_Climatic Change',
 93144659: 'GrayG_2014_A Review of Psychometric Data Analysis and Applications_Journal of Learning Analytics',
 2068614: 'UmehO_2009_Scanning the Parameter Space of Holographic Superconductors_Journal of High Energy Physics',
 41769583: 'BravoE_2013_Insights into thermonuclear supernovae from the incomplete Si-burning process_Astronomy & Astrophysics',
 30706837: 'LiH_2015_Safe Concurrency Introduction through Slicing_Proceedings of the 2015 Workshop on Partial Evaluation and Program Manipulation the 2015 Workshop on Partial Evaluation and Program Manipulation',
 141438903: 'BesoldT_2013_A Note on Tractability and Artificial Intelligence_Artificial General Intelligence',
 82002670: 'McdowallW_2012_Possible Hydrogen Transitions in the UK: Critical Uncertainties and Possible Decision Points_Energy Procedia',
 54009277: 'GuaraldiG_2015_Aging with HIV vs. HIV Seroconversion at Older Age: A Diverse Population with Distinct Comorbidity Profiles_PLOS ONE',
 55846727: 'MontresorA_2015_Methodological Bias Can Lead the Cochrane Collaboration to Irrelevance in Public Health Decision-Making Aims and Limits of the Cochrane Collaboration_PLOS Neglected Tropical Diseases',
 54847651: 'ByrdN_2014_Intuitive and Reflective Responses in Philosophy_nan',
 20652885: 'TownsendJ_2011_Taxon Sampling and the Optimal Rates of Evolution for Phylogenetic Inference_Systematic Biology',
 18493118: 'HolmesG_2013_Exploring the Relationship Between Local Support and the Success of Protected Areas_Conservation and Society',
 46115365: 'HerrendorfB_2015_Why is Measured Productivity so Low in Agriculture?_Review of Economic Dynamics',
 14579: 'MeteyardL_2009_THE RELATION BETWEEN CONTENT AND STRUCTURE IN LANGUAGE PRODUCTION: AN ANALYSIS OF SPEECH ERRORS IN SEMANTIC DEMENTIA_Brain and Language',
 56362892: 'GeigerI_2012_The bonding effects of relationship value and switching costs in industrial buyer-seller relationships: An investigation into role differences_Industrial Marketing Management',
 1683841: 'ShennanS_2008_Evolution in Archaeology_Annual Review of Anthropology',
 30320693: 'HortonJ_2014_For geographies of children, young people and popular culture_Geography Compass',
 39377931: 'GibbsM_2012_Design and implementation of pay for performance_nan',
 39369023: 'PycroftJ_2011_A tale of tails uncertainty and the social cost of carbon dioxide_Economics',
 82868732: 'BaveyeP_2015_Grand challenges in the research on soil processes_Frontiers in Environmental Science',
 56681767: 'BrisbinA_2012_PCAdmix: Principal Components-Based Assignment of Ancestry along Each Chromosome in Individuals with Admixed Ancestry from Two or More Populations_Human Biology',
 186286012: 'HerrmannovaD_2018_Unsupervised Identification of Study Descriptors in Toxicology Research: An Experimental Study_Proceedings of the Ninth International Workshop on Health Text Mining and Information Analysis the Ninth International Workshop on Health Text Mining and Information Analysis',
 81192723: 'BoeckhoutM_2015_Governing the research-care divide in clinical biobanking: Dutch perspectives_Life Sciences, Society and Policy',
 54009105: 'MapelliJ_2015_The Effect of Desflurane on Neuronal Communication at a Central Synapse_PLOS ONE',
 77101230: 'KatambaraZ_2014_Positioning Mbeya University of Science and Technology in Tanzania in the Systems of Innovation Perspective_Advances in Applied Sociology',
 46113415: 'ZerrahnA_2015_On the representation of demand-side management in power system models_Energy',
 82073833: 'ChakravartiA_2013_Distilling Pathophysiology from Complex Disease Genetics_Cell',
 81288194: 'DubrovinB_2009_On universality of critical behavior in the focusing nonlinear schrodinger, equation elliptic umbilic catastrophe and the tritronquee solution to the painleve-i equation_Journal of Nonlinear Science',
 30730382: 'CooperR_2015_Beyond Single-Level Accounts: The Role of Cognitive Architectures in Cognitive Scientific Explanation_Topics in Cognitive Science',
 29411898: 'KosterR_2015_Action and Valence Modulate Choice and Choice-Induced Preference Change_PLOS ONE',
 36208885: 'EvangelistiM_2014_Molecule-Based Magnetic Coolers: Measurement, Design and Application_NanoScience and Technology',
 1388501: 'FrancisL_2011_Work-Related Psychological Health Among Clergy Serving in The Presbyterian Church (USA): Testing the Idea of Balanced Affect_Review of Religious Research',
 30816019: 'KompusK_2015_Prevalence of auditory hallucinations in Norwegian adolescents: Results from a population-based study_Scandinavian Journal of Psychology',
 9634246: "CampbellF_2010_Identification of damaged proteins in human serum using modified Ehrlich's reagent to target protein-bound pyrroles_Analytical Biochemistry",
 42622455: 'KhwailehT_2015_Morpho-syntactic processing of Arabic plurals after aphasia: dissecting lexical meaning from morpho-syntax within word boundaries_Cognitive Neuropsychology',
 2193860: 'GuennouM_2011_Structural investigation of LaAlO 3 up to 63 GPa_Journal of Physics: Condensed Matter',
 81605842: 'IshiiR_2015_Everolimus improves behavioral deficits in a patient with autism associated with tuberous sclerosis: a case report_Neuropsychiatric Electrophysiology',
 35074230: 'PharoN_2013_Using search transitions to study searchers investment of effort experiences with client and server side logging_Lecture Notes in Computer Science',
 81739181: 'RuprechtU_2014_High photobiont diversity in the common European soil crust lichen Psora decipiens_Biodiversity and Conservation',
 74373376: 'GoergenM_2014_THE EMPLOYMENT CONSEQUENCES OF PRIVATE EQUITY ACQUISITIONS: THE CASE OF INSTITUTIONAL BUY OUTS_European Economic Review',
 144255832: 'ValdarniniR_2012_Hydrodynamic capabilities of an SPH code incorporating an artificial conductivity term with a gravity-based signal velocity_Astronomy & Astrophysics',
 24938638: 'HowarthI_2013_Emission-line stars in the LMC: the Armagh survey, and a metacatalogue (Research Note)_Astronomy & Astrophysics',
 82854048: 'FormanS_2015_Episodic eolian sand deposition in the past 4000 years in Cape Cod National Seashore, Massachusetts, USA in response to possible hurricane/storm and anthropogenic disturbances_Frontiers in Earth Science',
 1441362: 'PetersonR_2013_Social Memory and Ritual Performance_Journal of Social Archaeology',
 2085983: 'GerbierF_2009_Gauge fields for ultracold atoms in optical superlattices_New Journal of Physics',
 2188336: 'SasakuraN_2011_Tensor models and 3-ary algebras Tensor models and 3-ary algebras_Proceedings of Proceedings of the Corfu Summer Institute 2011 — PoS(CORFU2011) the Corfu Summer Institute 2011 — PoS(CORFU2011)',
 82152711: 'SchultzW_2011_Potential Vulnerabilities of Neuronal Reward, Risk, and Decision Mechanisms to Addictive Drugs_Neuron',
 5226895: 'BaxterS_2012_Interventions Using High-Technology Communication Devices: A State of the Art Review_Folia Phoniatrica et Logopaedica',
 81800077: 'RealeF_2014_Coronal Loops: Observations and Modeling of Confined Plasma_Living Reviews in Solar Physics',
 16246214: 'WolthersM_2012_Calcite surface structure and reactivity: molecular dynamics simulations and macroscopic surface modelling of the calcite-water interface_Physical Chemistry Chemical Physics',
 4893515: 'DeserS_2011_No Bel-Robinson Tensor for Quadratic Curvature Theories_Classical and Quantum Gravity',
 30780864: 'StaplesK_2015_Viral infection of human lung macrophages increases pdl1 expression via ifn_PLOS ONE',
 16229222: 'FaulknerA_2012_Comparing live to recorded speech in training the perception of spectrally shifted noise-vocoded speech_nan',
 9323912: 'LampardR_2012_Parental Characteristics, Family Structure and Occupational Attainment in Britain_Sociology',
 78263683: 'GurgiserW_2016_Comparing peasants perceptions of precipitation change with precipitation records in the tropical Callejon de Huaylas, Peru_Earth System Dynamics',
 81068384: 'BernheimJ_2014_Questions and Answers on the Belgian Model of Integral End-of-Life Care: Experiment? Prototype?_Journal of Bioethical Inquiry',
 2016794: 'KhavkineI_2008_Evaluation of new spin foam vertex amplitudes_Classical and Quantum Gravity',
 15459371: 'StraalenN_2008_Genomics technology for assessing soil pollution_nan',
 81905962: 'KociumakaT_2013_Fast Algorithm for Partial Covers in Words_Combinatorial Pattern Matching',
 81525093: 'KersteinA_2013_Hierarchical Parcel-Swapping Representation of Turbulent Mixing. Part 1. Formulation and Scaling Properties_Journal of Statistical Physics',
 82858263: 'TrinconeA_2014_Molecular fishing: marine oligosaccharides_Frontiers in Marine Science',
 30269693: 'AldenK_2015_Using argument notation to engineer biological simulations with increased confidence_Journal of The Royal Society Interface',
 37342138: 'BlanchardR_2011_Biofuels and biodiversity in South Africa_nan',
 81077179: 'TracyC_2008_Asymptotics in ASEP with Step Initial Condition_Communications in Mathematical Physics',
 41779783: 'FranchX_2014_Enabling Informed Decision Making through Mobile Technologies: A Challenge for Software Engineering_Communications in Computer and Information Science',
 9633126: 'LocktonD_2012_Social and interpersonal approaches to design for behaviour change_SSRN Electronic Journal',
 16272271: 'JonesA_2013_Re-conceptualising the space of markets the case of the 2007-9 global financial crisis_Geoforum',
 82162478: 'WatersD_2010_Exploring new indications for statins beyond atherosclerosis: Successes and setbacks_Journal of Cardiology',
 30874398: 'Toch-MarquardtM_2014_Occupational Class Inequalities in All-Cause and Cause-Specific Mortality among Middle-Aged Men in 14 European Populations during the Early 2000s_PLoS ONE',
 6560220: 'OlperA_2014_Political Reforms and Public Policies: Evidence from Agricultural Protection_SSRN Electronic Journal',
 81084827: 'GouT_2015_Do All Candle-Flame-Shaped Flares Have the Same Temperature Distribution?_Solar Physics',
 78485641: 'LuisM_2015_A Constructive Method and a Guided Hybrid GRASP for the Capacitated Multi-source Weber Problem in the Presence of Fixed Cost_Journal of Algorithms & Computational Technology',
 11291499: "HoffmannM_2012_Securitization of mortgage debt, domestic lending, and international risk sharing Securitization of Mortgage Debt, Domestic Lending and International Risk Sharing_Canadian Journal of Economics/Revue canadienne d'économique",
 42633036: 'HamiltonI_2015_Health effects of home energy efficiency interventions in England: a modelling study_BMJ Open',
 21748916: "ClaypoolM_2005_On the 802.11 Turbulence of Nintendo DS and Sony PSP Hand-held Network Games_Proceedings of 4th ACM SIGCOMM workshop on Network and system support for games - NetGames '05 4th ACM SIGCOMM workshop on Network and system support for games - NetGames '05",
 52040220: 'AbrahamsenM_2012_Sensemaking in business networks: Introducing dottograms to analyse network changes_Industrial Marketing Management',
 12030665: 'SarkarP_2013_Does employment protection lead to unemployment? A panel data analysis of OECD countries, 1990-2008_nan',
 20485596: 'MavranezouliI_2014_The cost-effectiveness of supported employment for adults with autism in the United Kingdom_Autism',
 78076775: 'DawsonE_2014_Not designed for us - How science museums and science centers socially exclude low income minority ethnic groups_Science Education',
 13505261: 'KesserwaniG_2013_Topography discretization techniques for Godunov-type shallow water numerical models: a comparative study_Journal of Hydraulic Research',
 52040210: 'BenitoG_2012_Governance costs in foreign direct investments: A MNC headquarters challenge_Journal of International Management',
 82867877: 'JellingerK_2013_Pathology and pathogenesis of vascular cognitive impairment a critical update_Frontiers in Aging Neuroscience',
 1385821: 'NicoliniD_2011_Policy and practice in the use of root cause analysis to investigate clinical adverse events: Mind the gap_Social Science & Medicine',
 82867364: 'CorballisM_2013_Wandering tales: evolutionary origins of mental time travel and language_Frontiers in Psychology',
 187054721: 'QasrawiH_2018_Towards Sustainable Self-Compacting Concrete: Effect of Recycled Slag Coarse Aggregate on the Fresh Properties of SCC_Advances in Civil Engineering'}

In [25]:
filename_to_core_id = {value: key for key, value in core_id_to_filename.items()}


In [26]:
filename_to_core_id = {key + '.tei.xml': value for key, value in filename_to_core_id.items()}

In [27]:
filename_to_core_id

{'HohensinnerS_2013_Two steps back, one step forward: reconstructing the dynamic Danube riverscape under human influence in Vienna_Water History.tei.xml': 81153632,
 'JensenO_2013_Reproducibility of tender point examination in chronic low back pain patients as measured by intrarater and inter-rater reliability and agreement: a validation study_BMJ Open.tei.xml': 50694047,
 'RobinsonE_2011_A Simple Threshold Rule Is Sufficient to Explain Sophisticated Collective Decision-Making_PLoS ONE.tei.xml': 5225915,
 'HeatherN_2014_Interpreting Null Findings from Trials of Alcohol Brief Interventions_Frontiers in Psychiatry.tei.xml': 82875399,
 'Funck-BrentanoC_2013_Effects of rabeprazole on the antiplatelet effects and pharmacokinetics of clopidogrel in healthy volunteers_Archives of Cardiovascular Diseases.tei.xml': 82218905,
 'BakerD_2015_Brain networks underlying bistable perception_NeuroImage.tei.xml': 30272741,
 'ValleL_2015_Official Statistics Data Integration for Enhanced Information Quali

In [56]:
papers['core_id'] = papers['filename_xml'].map(filename_to_core_id)

In [71]:
papers.loc[6, 'core_id'] = 187052580
papers.loc[37, 'core_id'] = 21748916
papers.loc[65, 'core_id'] = 82854048
papers.loc[77, 'core_id'] = 81765890
papers.loc[92, 'core_id'] = 186286012
papers.loc[94, 'core_id'] = 11291499
papers.loc[109, 'core_id'] = 211327317
papers.loc[133, 'core_id'] = 76526352
papers.loc[188, 'core_id'] = 9636910

In [13]:
papers.core_id.nunique()

229

In [88]:
df = pd.merge(left = df, right = papers[['core_id', 'article_id']], on = 'core_id', how = 'left')

In [15]:
sent = pd.read_csv('data/causeminer_data/sentences.csv')

In [19]:
sent = pd.merge(left = sent, right = papers[['article_id', 'core_id']], on = 'article_id', how = 'left')

## matching references

In [6]:
import recordlinkage as rl
from recordlinkage.index import Block

In [92]:
# what records can we match?

In [7]:
df1 = df[['unique_id', 'core_id', 'cited_title', 'cited_author', 'cited_publication_date', 'cited_publication_info']]

In [128]:
dict = {'cited_title': 'title',
        'cited_author': 'authors',
        'cited_publication_date': 'year',
       'cited_publication_info': 'journal'}

In [9]:
df1.rename(columns=dict,
          inplace=True)

In [10]:
df1['authors'] = df1['authors'].str.replace(r'["\[\]]', '')

In [11]:
df1.year = df1.year.fillna(0).astype(int)

In [12]:
df1['year'] = df1.year.replace(0, np.nan)

In [13]:
df2 = ref[['reference_key', 'authors', 'year', 'title', 'journal', 'core_id']]

In [14]:
df2.year = df2.year.fillna(0).astype(int)

In [15]:
df2['year'] = df2.year.replace(0, np.nan)

In [16]:
indexer = rl.Index()
indexer.add(Block(left_on = 'core_id', right_on = None))

In [17]:
candidate_pairs = indexer.index(df1, df2)

In [18]:
len(candidate_pairs)

332304

In [19]:
comparer = rl.Compare()
comparer.string(right_on ='title',left_on = 'title', method='jarowinkler', label='title')
comparer.exact(right_on ='year', left_on ='year', label='year')
comparer.string(right_on ='authors',left_on ='authors', method = 'jarowinkler', label='authors')
comparer.string(right_on ='journal', left_on ='journal', method='jarowinkler', label='journal')
#comparer.string('cited_doi', 'doi', label='doi')


<Compare>

In [20]:
features = comparer.compute(candidate_pairs, df1, df2)


In [60]:
features

title  year   authors   journal
0    578    0.660978     0  0.430556  0.576441
     908    0.523685     0  0.505556  0.525402
     975    0.625462     0  0.501068  0.478047
     1033   0.626129     0  0.485815  0.527675
     1319   0.000000     0  0.449074  0.000000
...              ...   ...       ...       ...
3999 11659  0.645977     0  0.383191  0.546032
     11663  0.613390     0  0.426852  0.000000
     11664  0.676737     0  0.407115  0.000000
     12360  0.488579     0  0.381481  0.000000
     13885  0.661435     0  0.417989  0.000000

[332304 rows x 4 columns]

In [21]:
features = features.reset_index()#.drop_duplicates('level_0', keep= 'first')

In [22]:
# Initialize counters
total_zeros = 0
nan_zeros = 0

# Iterate over feature rows
for idx, row in features.iterrows():
    # get original records
    record_df1 = df1.loc[row['level_0']]
    record_df2 = df2.loc[row['level_1']]

    # Iterate over comparison columns
    for col in ['title', 'year', 'authors', 'journal']:
        # Check if comparison result is 0
        if row[col] == 0:
            # Increment total zero counter
            total_zeros += 1

            # Check if either corresponding value is NaN
            if pd.isna(record_df1[col]) or pd.isna(record_df2[col]):
                # Increment NaN zero counter
                nan_zeros += 1

                # Replace 0 with NaN in features DataFrame
                features.at[idx, col] = np.nan

# Calculate and print ratio
if total_zeros > 0:  # avoid division by zero
    ratio = nan_zeros / total_zeros
    print(f"The ratio of 0 similarity scores due to NaN values to total 0 similarity scores is {ratio}")
else:
    print("No zero similarity scores found.")

The ratio of 0 similarity scores due to NaN values to total 0 similarity scores is 0.28526670669622933


In [23]:
features['average_similarity'] = features[['title', 'year', 'authors', 'journal']].mean(axis=1)

In [24]:
features = features.loc[(features.title > 0.5) & (features.average_similarity > 0.5)].sort_values('average_similarity', ascending = False).drop_duplicates('level_0', keep = 'first')

In [114]:
features = features.drop_duplicates('level_1', keep= 'first')

In [31]:
features.level_1.duplicated().sum()

291

In [36]:
features = features.drop_duplicates('level_1', keep = 'first')

In [29]:
df1.loc[884]

unique_id                     CC885
core_id                    82899510
title            Intentional Stance
authors                 D C Dennett
year                         1976.0
journal      Psychological Medicine
Name: 884, dtype: object

In [30]:
df2.loc[3102]

reference_key                    69_b36
authors                     Dennett, D.
year                             1987.0
title            the intentional stance
journal                             NaN
core_id                        82899510
Name: 3102, dtype: object

In [41]:
merged_df = pd.merge(features, df2[['reference_key']], left_on='level_1', right_index=True, how='left')

In [42]:
merged_df = merged_df[['level_0', 'reference_key']]

In [43]:
merged_df

,level_0,reference_key
220864,2994,183_b1
330321,3968,41_b79
159199,2273,80_b129
129792,1972,50_b55
99550,1538,136_b36
...,...,...
210998,2763,55_b42
208089,2700,24_b52
69141,884,69_b36
87989,1337,47_b3


In [45]:
df1 = pd.merge(left = df1, right = merged_df, left_index=True, right_on='level_0', how='left')

In [49]:
df1

,unique_id,core_id,title,authors,year,journal,level_0,reference_key
NaN,CC1,81153632,The English Village Community Examined in its ...,Frederic Seebohm,NaN,The English Village Community Examined in its ...,0,NaN
123.0,CC2,81153632,Die italienischen Begründer der Wiener Donauka...,Friedrich Slezak,1977.0,Der Donauraum,1,96_b60
182.0,CC3,81153632,Wien im Bild historischer Karten Die Entwicklu...,Ferdinand Opll,2004.0,Jahrhunderts,2,96_b48
231.0,CC4,81153632,The hydraulic geometry of stream channels and ...,"Luna Bergere Leopold, Thomas Maddock",1953.0,Geological Survey Professional Paper,3,96_b37
273.0,CC5,81153632,From memory to written record,M T Clanchy,1993.0,From memory to written record,4,96_b10
...,...,...,...,...,...,...,...,...
1232.0,CC3996,81153632,Modelling channel evolution and floodplain mor...,A D Howard,1996.0,Modelling channel evolution and floodplain mor...,3995,96_b32
1311.0,CC3997,81153632,Atlas of the Danube River Vienna A history of ...,"P Mohilla, F Michlmayr",1996.0,Donauatlas Wien: Geschichte der Donauregulieru...,3996,96_b45
1387.0,CC3998,81153632,Alte Grenzen im Wiener Raum,F Opll,1986.0,"Kommentare zum Historischen Atlas von Wien, Vi...",3997,96_b47
1458.0,CC3999,81153632,Avulsive channel systems: characteristics and ...,"Keith Richards, Shobhit Chandra, Peter Friend",1993.0,"Geological Society, London, Special Publications",3998,96_b54


In [51]:
df1.isna().sum()

unique_id          0
core_id            0
title              0
authors            0
year             112
journal           72
level_0            0
reference_key    449
dtype: int64

In [52]:
full = df1.merge(df2, on='reference_key', how='left', suffixes=('', '_y'))

In [53]:
for column in ['authors', 'year', 'title', 'journal']:
    full[column].fillna(full[column+'_y'], inplace=True)
    full.drop(column+'_y', axis=1, inplace=True)

In [61]:
full = full.drop(columns = ['level_0', 'core_id_y'])

In [79]:
mask = (df1.journal.isna() == True) & (full.journal.isna() == False)

In [82]:
full.loc[mask, 'journal'] = full.loc[mask, 'journal'].str.title()


In [104]:
full.equals(df1)

False

In [106]:
df_all = pd.concat([df1, full])

# Drop duplicates. The remaining rows are those that have been changed.
df_changed = df_all.drop_duplicates(keep=False)

# As the above operation concatenates both unchanged and changed rows,
# you need to do it again to get only the changed rows
df_changed = df_changed.drop_duplicates()

In [107]:
df_changed

,unique_id,core_id,title,authors,year,journal,reference_key
203,CC204,50694980,Recovery of Muscle Strength Following Arthrosc...,Matthews,NaN,NaN,204_b2
204,CC205,50694980,Impact of knee extensor strength deficits on s...,Moffet,NaN,NaN,204_b3
205,CC206,50694980,Isokinetic rehabilitation after arthroscopic m...,St-Pierre,NaN,NaN,204_b4
256,CC257,82865416,A Physiological Model of a Circannual Oscillator,Macgregor,NaN,NaN,166_b55
282,CC283,76981266,National Institute for Health and Clinical Exc...,Moussavi,NaN,NaN,84_b23
...,...,...,...,...,...,...,...
3841,CC3842,78076775,Public attitudes to science 2011 London Depart...,Ipsos,2011.0,NaN,48_b56
3856,CC3857,52040210,The Economic Institutions of Capitalism Firms ...,Oliver E. Williamson,2010.0,"Das Summa Summarum des Management, New York",16_b8
3925,CC3926,1385821,Organizational Learning: Affirming an Oxymoron,"Karl E. Weick, Frances Westley",1996.0,"Managing Organizations: Current Issues, London",155_b32
3971,CC3972,82867364,A pragmatic perspective on the evolution of la...,"Dan Sperber, Gloria Origgi, Deirdre Wilson, Da...",2010.0,Meaning and Relevance,41_b75


In [108]:
full

,unique_id,core_id,title,authors,year,journal,reference_key
0,CC1,81153632,The English Village Community Examined in its ...,Frederic Seebohm,NaN,The English Village Community Examined in its ...,NaN
1,CC2,81153632,Die italienischen Begründer der Wiener Donauka...,Friedrich Slezak,1977.0,Der Donauraum,96_b60
2,CC3,81153632,Wien im Bild historischer Karten Die Entwicklu...,Ferdinand Opll,2004.0,Jahrhunderts,96_b48
3,CC4,81153632,The hydraulic geometry of stream channels and ...,"Luna Bergere Leopold, Thomas Maddock",1953.0,Geological Survey Professional Paper,96_b37
4,CC5,81153632,From memory to written record,M T Clanchy,1993.0,From memory to written record,96_b10
...,...,...,...,...,...,...,...
3995,CC3996,81153632,Modelling channel evolution and floodplain mor...,A D Howard,1996.0,Modelling channel evolution and floodplain mor...,96_b32
3996,CC3997,81153632,Atlas of the Danube River Vienna A history of ...,"P Mohilla, F Michlmayr",1996.0,Donauatlas Wien: Geschichte der Donauregulieru...,96_b45
3997,CC3998,81153632,Alte Grenzen im Wiener Raum,F Opll,1986.0,"Kommentare zum Historischen Atlas von Wien, Vi...",96_b47
3998,CC3999,81153632,Avulsive channel systems: characteristics and ...,"Keith Richards, Shobhit Chandra, Peter Friend",1993.0,"Geological Society, London, Special Publications",96_b54


In [118]:
df = pd.merge(left = df, right = full[['unique_id', 'reference_key']], on = 'unique_id', how = 'left')

In [85]:
rename_dict = {
    'title': 'cited_title',
    'authors': 'cited_author',
    'year': 'cited_publication_date',
    'journal': 'cited_publication_info'
}

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_id                 4000 non-null   object 
 1   core_id                   4000 non-null   int64  
 2   citation_offset           3995 non-null   float64
 3   total_doc_length          4000 non-null   int64  
 4   section_info              4000 non-null   object 
 5   citing_title              4000 non-null   object 
 6   citing_author             4000 non-null   object 
 7   citing_publication_info   4000 non-null   int64  
 8   citing_abstract           3992 non-null   object 
 9   cited_title               4000 non-null   object 
 10  cited_author              4000 non-null   object 
 11  cited_abstract            3040 non-null   object 
 12  cited_doi                 3646 non-null   object 
 13  cited_publication_date    3988 non-null   float64
 14  cited_pu

In [126]:
full

,unique_id,core_id,cited_title,cited_author,cited_publication_date,cited_publication_info,reference_key
0,CC1,81153632,The English Village Community Examined in its ...,Frederic Seebohm,NaN,The English Village Community Examined in its ...,NaN
1,CC2,81153632,Die italienischen Begründer der Wiener Donauka...,Friedrich Slezak,1977.0,Der Donauraum,96_b60
2,CC3,81153632,Wien im Bild historischer Karten Die Entwicklu...,Ferdinand Opll,2004.0,Jahrhunderts,96_b48
3,CC4,81153632,The hydraulic geometry of stream channels and ...,"Luna Bergere Leopold, Thomas Maddock",1953.0,Geological Survey Professional Paper,96_b37
4,CC5,81153632,From memory to written record,M T Clanchy,1993.0,From memory to written record,96_b10
...,...,...,...,...,...,...,...
3995,CC3996,81153632,Modelling channel evolution and floodplain mor...,A D Howard,1996.0,Modelling channel evolution and floodplain mor...,96_b32
3996,CC3997,81153632,Atlas of the Danube River Vienna A history of ...,"P Mohilla, F Michlmayr",1996.0,Donauatlas Wien: Geschichte der Donauregulieru...,96_b45
3997,CC3998,81153632,Alte Grenzen im Wiener Raum,F Opll,1986.0,"Kommentare zum Historischen Atlas von Wien, Vi...",96_b47
3998,CC3999,81153632,Avulsive channel systems: characteristics and ...,"Keith Richards, Shobhit Chandra, Peter Friend",1993.0,"Geological Society, London, Special Publications",96_b54


In [129]:
full.rename(columns=dict, inplace=True)

In [136]:
# Assuming 'df' and 'full' are your dataframes

# Perform a left join on 'unique_id'
merged_df = df.merge(full[['unique_id', 'title', 'year', 'authors', 'journal']], on='unique_id', how='left')

# Fill NaN values in 'df' with values from 'full'
merged_df['cited_title'].fillna(merged_df['title'], inplace=True)
merged_df['cited_author'].fillna(merged_df['authors'], inplace=True)
merged_df['cited_publication_date'].fillna(merged_df['year'], inplace=True)
merged_df['cited_publication_info'].fillna(merged_df['journal'], inplace=True)

# Drop the original columns from 'full'
merged_df.drop(['title', 'authors', 'year', 'journal'], axis=1, inplace=True)


In [146]:
df.loc[44]

unique_id                                                                CC45
core_id                                                               5225915
citation_offset                                                       42319.0
total_doc_length                                                        51347
section_info                {"Introduction": 2851, "Methods": 9661, "Resul...
citing_title                A Simple Threshold Rule Is Sufficient to Expla...
citing_author                                                   Elva Robinson
citing_publication_info                                                  2011
citing_abstract             Decision-making animals can use slow-but-accur...
cited_title                 Dancing bees tune both duration and rate of wa...
cited_author                ["Thomas D. Seeley", "Alexander S. Mikheyev", ...
cited_abstract              For more than 50 years, investigators of the h...
cited_doi                                               10.1007/

## looking for citation data

In [176]:
cite.loc[cite.reference_key == '1_b10']

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey
7,1,1_697_755,1_4,1,Introduction,NaN,1_b10,wilmot_e_sedentary_time_in_adults_and_the_asso...,diabetologia,aadland_e_reliability_of_objectively_measured_...


In [179]:
test_cite = cite

In [180]:
cite['section_number'], cite['sentence_start'], cite['sentence_end'] = cite['sentence_id'].str.split('_').str

In [182]:
cite['section_number'] = cite['section_number'].astype(int)
cite['sentence_start'] = cite['sentence_start'].astype(int)
cite['sentence_end'] = cite['sentence_end'].astype(int)

In [186]:
cite = cite.drop(columns = 'section_number')

In [187]:
cite

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey,sentence_start,sentence_end
0,1,1_456_509,1_3,1,Introduction,NaN,1_b0,hutcheon_j_random_measurement_error_and_regres...,bmj,aadland_e_reliability_of_objectively_measured_...,456,509
1,1,1_515_540,1_4,1,Introduction,NaN,1_b1,trost_s_conducting_accelerometerbased_activity...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,515,540
2,1,1_540_593,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,540,593
3,1,1_540_593,1_4,1,Introduction,NaN,1_b7,hart_t_how_many_days_of_monitoring_predict_phy...,int journal of behav nutr phys act,aadland_e_reliability_of_objectively_measured_...,540,593
4,1,1_593_625,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,593,625
...,...,...,...,...,...,...,...,...,...,...,...,...
19421,229,229_3998_4037,229_39,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",3998,4037
19422,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b28,knowlton_b_the_information_acquired_during_art...,journal of experimental psychology learning me...,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551
19423,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b37,redington_m_transfer_in_artificial_grammar_lea...,journal of experimental psychology general,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551
19424,229,229_4739_4768,229_50,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4739,4768


In [189]:
grouped = cite.groupby(['article_id', 'section_nr'])

In [190]:
section_starts = grouped['sentence_start'].min()
section_ends = grouped['sentence_end'].max()

In [191]:
section_lengths = section_ends - section_starts

In [192]:
section_lengths

article_id  section_nr
1           1              960
            3              176
            4              461
            11            1710
            12              46
                          ... 
229         2              204
            4               31
            5              138
            6              589
            8              880
Length: 1523, dtype: int64

In [193]:
section_lengths = section_lengths.reset_index()
section_lengths.columns = ['article_id', 'section_nr', 'section_length']

In [195]:
cite = pd.merge(cite, section_lengths, on=['article_id', 'section_nr'], how='left')

In [198]:
cite

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey,sentence_start,sentence_end,section_length
0,1,1_456_509,1_3,1,Introduction,NaN,1_b0,hutcheon_j_random_measurement_error_and_regres...,bmj,aadland_e_reliability_of_objectively_measured_...,456,509,960
1,1,1_515_540,1_4,1,Introduction,NaN,1_b1,trost_s_conducting_accelerometerbased_activity...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,515,540,960
2,1,1_540_593,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,540,593,960
3,1,1_540_593,1_4,1,Introduction,NaN,1_b7,hart_t_how_many_days_of_monitoring_predict_phy...,int journal of behav nutr phys act,aadland_e_reliability_of_objectively_measured_...,540,593,960
4,1,1_593_625,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,593,625,960
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19421,229,229_3998_4037,229_39,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",3998,4037,880
19422,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b28,knowlton_b_the_information_acquired_during_art...,journal of experimental psychology learning me...,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551,880
19423,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b37,redington_m_transfer_in_artificial_grammar_lea...,journal of experimental psychology general,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551,880
19424,229,229_4739_4768,229_50,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4739,4768,880


In [199]:
cite['sentence_middle'] = cite[['sentence_start', 'sentence_end']].mean(axis=1)

In [201]:
cite['relative_position'] = cite['sentence_middle'] / cite['section_length']

In [203]:
cite

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey,sentence_start,sentence_end,section_length,sentence_middle,relative_position
0,1,1_456_509,1_3,1,Introduction,NaN,1_b0,hutcheon_j_random_measurement_error_and_regres...,bmj,aadland_e_reliability_of_objectively_measured_...,456,509,960,482.5,0.502604
1,1,1_515_540,1_4,1,Introduction,NaN,1_b1,trost_s_conducting_accelerometerbased_activity...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,515,540,960,527.5,0.549479
2,1,1_540_593,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,540,593,960,566.5,0.590104
3,1,1_540_593,1_4,1,Introduction,NaN,1_b7,hart_t_how_many_days_of_monitoring_predict_phy...,int journal of behav nutr phys act,aadland_e_reliability_of_objectively_measured_...,540,593,960,566.5,0.590104
4,1,1_593_625,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,593,625,960,609.0,0.634375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19421,229,229_3998_4037,229_39,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",3998,4037,880,4017.5,4.565341
19422,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b28,knowlton_b_the_information_acquired_during_art...,journal of experimental psychology learning me...,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551,880,4505.5,5.119886
19423,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b37,redington_m_transfer_in_artificial_grammar_lea...,journal of experimental psychology general,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551,880,4505.5,5.119886
19424,229,229_4739_4768,229_50,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4739,4768,880,4753.5,5.401705


In [204]:
cite['relative_middle'] = cite['sentence_middle'] - cite['sentence_start']

In [206]:
cite['relative_position'] = cite['relative_middle'] / cite['section_length']

In [210]:
cite

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey,sentence_start,sentence_end,section_length,sentence_middle,relative_position,relative_middle
0,1,1_456_509,1_3,1,Introduction,NaN,1_b0,hutcheon_j_random_measurement_error_and_regres...,bmj,aadland_e_reliability_of_objectively_measured_...,456,509,960,482.5,0.027604,26.5
1,1,1_515_540,1_4,1,Introduction,NaN,1_b1,trost_s_conducting_accelerometerbased_activity...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,515,540,960,527.5,0.013021,12.5
2,1,1_540_593,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,540,593,960,566.5,0.027604,26.5
3,1,1_540_593,1_4,1,Introduction,NaN,1_b7,hart_t_how_many_days_of_monitoring_predict_phy...,int journal of behav nutr phys act,aadland_e_reliability_of_objectively_measured_...,540,593,960,566.5,0.027604,26.5
4,1,1_593_625,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,593,625,960,609.0,0.016667,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19421,229,229_3998_4037,229_39,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",3998,4037,880,4017.5,0.022159,19.5
19422,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b28,knowlton_b_the_information_acquired_during_art...,journal of experimental psychology learning me...,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551,880,4505.5,0.051705,45.5
19423,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b37,redington_m_transfer_in_artificial_grammar_lea...,journal of experimental psychology general,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4460,4551,880,4505.5,0.051705,45.5
19424,229,229_4739_4768,229_50,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",4739,4768,880,4753.5,0.016477,14.5


In [214]:
# Compute section_start and section_end for each section of each article
section_data = cite.groupby(['article_id', 'section_nr']).agg(
    section_start_new=('sentence_start', 'min'),
    section_end_new=('sentence_end', 'max')
).reset_index()

# Compute section_length based on these new values
section_data['section_length_new'] = section_data['section_end_new'] - section_data['section_start_new']

# Merge this data back into the main dataframe
cite = pd.merge(cite, section_data, on=['article_id', 'section_nr'], how='left')

# Compute "zero-based" sentence start and end, and find the sentence middle based on these
cite['sentence_start_zero'] = cite['sentence_start'] - cite['section_start_new']
cite['sentence_end_zero'] = cite['sentence_end'] - cite['section_start_new']
cite['sentence_middle_zero'] = (cite['sentence_end_zero'] + cite['sentence_start_zero']) / 2

# Now find the relative position
cite['relative_position'] = cite['sentence_middle_zero'] / cite['section_length_new']


In [219]:
cite.relative_position.min()

0.0012387534228713

## anotated data sentence similarity

In [8]:
ann = pd.read_excel('anotated.xlsx')

In [14]:
ann.most_similar_sentence_id.isna().sum()

860

In [20]:
ann

,unique_id,reference_key,citation_context,most_similar_sentence,similarity_score,most_similar_sentence_id
0,CC674,177_b22,Several systems of set theory for the construc...,NaN,0.0,NaN
1,CC1271,56_b34,"Thus, on the basis of these non-linear changes...",NaN,0.0,NaN
2,CC1401,216_b41,"However, the clustering of businesses in e.g. ...",NaN,0.0,NaN
3,CC1716,141_b41,This ensured that the full posterior distribut...,NaN,0.0,NaN
4,CC1757,102_b43,This concept was first proposed by #CITATION_TAG.,NaN,0.0,NaN
...,...,...,...,...,...,...
3995,CC3971,NaN,What has been termed episodic foresight (Sudde...,NaN,NaN,NaN
3996,CC3977,NaN,Some have argued against the gestural theory o...,NaN,NaN,NaN
3997,CC3987,NaN,In a recent analysis of the neural mechanisms ...,NaN,NaN,NaN
3998,CC3994,NaN,"Together with augmented sediment loads, it add...",NaN,NaN,NaN


# enriching by positions

## position within sentence

In [55]:
def calculate_citation_position(sentence):
    citation_tag_index = sentence.find('#CITATION_TAG')
    if citation_tag_index == -1:
        return None  # Return None if #CITATION_TAG is not found
    return citation_tag_index / len(sentence)  # Return position as a value between 0 and 1

In [56]:
df['cite_pos_in_sent'] = df.citation_context.apply(calculate_citation_position)

## position of sentence in article

In [58]:
df['sent_pos_in_article'] = df.citation_offset / df.total_doc_length

## position of sentence within section (to be continued)

In [238]:
merged_df = df.dropna(subset = 'reference_key')

In [242]:
pd.merge(left = merged_df, right = cite[['reference_key', 'relative_position']], on = 'reference_key', how = 'left')

,unique_id,core_id,citation_offset,total_doc_length,section_info,citing_title,citing_author,citing_publication_info,citing_abstract,cited_title,...,co_mentions,citation_class_label,citation_influence_label,cite_pos_in_sent,sent_pos_in_article,article_id,citing_publication_dat,reference_key,diff_publication_date,relative_position
0,CC2,81153632,23562.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,Die italienischen Begründer der Wiener Donauka...,...,1.0,0,1,0.929245,0.394515,96,2013,96_b60,36.0,0.776542
1,CC3,81153632,22736.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,Wien im Bild historischer Karten Die Entwicklu...,...,2.0,4,1,0.912281,0.380684,96,2013,96_b48,9.0,0.731020
2,CC4,81153632,46410.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,The hydraulic geometry of stream channels and ...,...,0.0,0,0,0.484615,0.777075,96,2013,96_b37,60.0,0.303303
3,CC5,81153632,4191.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,From memory to written record,...,0.0,0,0,0.888060,0.070173,96,2013,96_b10,20.0,NaN
4,CC7,81153632,6086.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,Methods for spatial and temporal land use and ...,...,7.0,0,0,0.171123,0.101902,96,2013,96_b15,16.0,0.331492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6652,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.017324
6653,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.089990
6654,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.284408
6655,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.655919


In [250]:
cite.loc[(cite.reference_key.duplicated() == True) & (cite.sentence_id.duplicated() == True)].sort_values('sentence_id')

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey,...,relative_middle,section_start,section_end,section_length_y,sentence_start_zero,sentence_end_zero,sentence_middle_zero,section_start_new,section_end_new,section_length_new
7867,100,100_2568_2633,100_14,2,Part 1 : What Children 's Popular Culture Means,NaN,100_b41,kline_s_out_of_the_garden_toys_and_childrens_c...,book,horton_j_for_geographies_of_children_young_peo...,...,32.5,982,7719,6737,1586,1651,1618.5,982,7719,6737
7873,100,100_2707_2780,100_15,2,Part 1 : What Children 's Popular Culture Means,NaN,100_b4,bickford_t_kidz_bop_tweens_and_childhood_music...,consumers commodities and consumption,horton_j_for_geographies_of_children_young_peo...,...,36.5,982,7719,6737,1725,1798,1761.5,982,7719,6737
7876,100,100_3124_3225,100_17,2,Part 1 : What Children 's Popular Culture Means,NaN,100_b67,saltmarsh_s_becoming_economic_subjects_agency_...,discourse,horton_j_for_geographies_of_children_young_peo...,...,50.5,982,7719,6737,2142,2243,2192.5,982,7719,6737
7927,101,101_1372_1444,101_11,3,Cross - identification metacatalogue,NaN,101_b23,reid_w__2012_mnras,mnras,howarth_i_emission_line_stars_in_the_lmc_the_a...,...,36.0,1296,1593,297,76,148,112.0,1296,1593,297
7934,101,101_1836_1905,101_14,4,"2012 ; Neugent et al . 2012 ) , 3 although no ...",NaN,101_b8,evans_c__2011_aa,aa,howarth_i_emission_line_stars_in_the_lmc_the_a...,...,34.5,1761,2296,535,75,144,109.5,1761,2296,535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,9,9_1213_1263,9_13,2,Related work,NaN,9_b0,allen_m__1987_book,book,baa_p_application_of_pfortran_and_co_array_for...,...,25.0,1034,1391,357,179,229,204.0,1034,1391,357
625,9,9_1361_1391,9_14,2,Related work,NaN,9_b6,clark_t_experiences_in_dataparallel_programmin...,journal of scientific programming,baa_p_application_of_pfortran_and_co_array_for...,...,15.0,1034,1391,357,327,357,342.0,1034,1391,357
631,9,9_1931_1991,9_25,4,Parallelization strategy,NaN,9_b8,clark_t_parallelizing_molecular_dynamics_using...,book,baa_p_application_of_pfortran_and_co_array_for...,...,30.0,1931,2413,482,0,60,30.0,1931,2413,482
632,9,9_1931_1991,9_25,4,Parallelization strategy,NaN,9_b7,clark_t_evaluating_parallel_languages_for_mole...,book,baa_p_application_of_pfortran_and_co_array_for...,...,30.0,1931,2413,482,0,60,30.0,1931,2413,482


In [234]:
df.reference_key.isna().sum()

449

In [254]:
cite.loc[cite.reference_key == '1_b5']

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey,...,relative_middle,section_start,section_end,section_length_y,sentence_start_zero,sentence_end_zero,sentence_middle_zero,section_start_new,section_end_new,section_length_new
2,1,1_540_593,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,26.5,456,1416,960,84,137,110.5,456,1416,960
4,1,1_593_625,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,16.0,456,1416,960,137,169,153.0,456,1416,960
19,1,1_991_1020,1_5,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,14.5,456,1416,960,535,564,549.5,456,1416,960
29,1,1_1378_1416,1_6,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,19.0,456,1416,960,922,960,941.0,456,1416,960
49,1,1_3609_3640,1_30,11,Discussion,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,15.5,3502,5212,1710,107,138,122.5,3502,5212,1710
65,1,1_4122_4187,1_31,11,Discussion,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,32.5,3502,5212,1710,620,685,652.5,3502,5212,1710
77,1,1_4434_4466,1_31,11,Discussion,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,16.0,3502,5212,1710,932,964,948.0,3502,5212,1710
85,1,1_5160_5212,1_34,11,Discussion,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,26.0,3502,5212,1710,1658,1710,1684.0,3502,5212,1710


In [21]:
import ast

In [22]:
def contains_target(s):
    # parse the string into a list
    lst = ast.literal_eval(s)
    return '1_b5' in lst

# Use the apply function to create a mask
mask = sent['references'].apply(contains_target)

# Use the mask to select the rows
result = sent[mask]


In [30]:
result

,article_id,sentence_id,para_id,sentence_original,sentence,sentence_ent_embed,section_nr,last_section_title,last_subsection_title,sentence_type,references,number_of_citations,core_id
27,1,1_540_593,1_4,"However, estimates of how many days of monitoring that should be included to obtain a reliable result vary considerably between studies [3-7, 2, 8], and might also vary between outcome variables of interest [6, 8] .","However , estimates of how many days of monitoring that should be included to obtain a reliable result vary considerably between studies [ 3 - 7 , 2 , 8 ] , and might also vary between outcome variables of interest START_CITE [ 6 , END_CITE CITE_b5 START_CITE 8 ] END_CITE CITE_b7 .","However , estimates of how many days of monitoring that should be included to obtain a reliable result vary considerably between studies [ 3 - 7 , 2 , 8 ] , and might also vary between [dependent_variable_ENTITY] of interest START_CITE [ 6 , END_CITE CITE_b5 START_CITE 8 ] END_CITE CITE_b7 .",1,Introduction,NaN,PARAGRAPH,"['1_b5', '1_b7']",2,52129671
28,1,1_593_625,1_4,"According to Matthews et al [6] , inclusion of more days may be needed to arrive at reliable estimates (intraclass correlation coefficient (ICC) !","According to Matthews et al START_CITE [ 6 ] END_CITE CITE_b5 , inclusion of more days may be needed to arrive at reliable estimates ( intraclass correlation coefficient ( intraclass-correlation coefficients ) !","According to Matthews et al START_CITE [ 6 ] END_CITE CITE_b5 , inclusion of more days may be needed to arrive at reliable estimates ( intraclass [Pearson_correlation_coefficient_ENTITY] ( intraclass-[Pearson_correlation_coefficient_ENTITY] ) !",1,Introduction,NaN,PARAGRAPH,['1_b5'],1,52129671
40,1,1_991_1020,1_5,"This cutoff differs substantially from that used to define ""inactivity"" (<500 cpm) applied by Matthews et al [6] .","This cutoff differs substantially from that used to define "" inactivity "" ( < 500 cpm ) applied by Matthews et al START_CITE [ 6 ] END_CITE CITE_b5 .","This cutoff differs substantially from that used to define "" inactivity "" ( < 500 cpm ) applied by Matthews et al START_CITE [ 6 ] END_CITE CITE_b5 .",1,Introduction,NaN,PARAGRAPH,['1_b5'],1,52129671
53,1,1_1378_1416,1_6,"Based on previous findings [8, 6] , we hypothesized that a longer monitoring period would be needed to obtain reliable estimates of SED, compared to other outcomes.","Based on previous findings START_CITE [ 8 , END_CITE CITE_b7 START_CITE 6 ] END_CITE CITE_b5 , we hypothesized that a longer monitoring period would be needed to obtain reliable estimates of sedentary behavior , compared to other outcomes .","Based on previous findings START_CITE [ 8 , END_CITE CITE_b7 START_CITE 6 ] END_CITE CITE_b5 , we hypothesized that a longer monitoring period would be needed to obtain reliable estimates of sedentary behavior , compared to other outcomes .",1,Introduction,NaN,PARAGRAPH,"['1_b7', '1_b5']",2,52129671
196,1,1_3609_3640,1_30,"Consistent with previous studies [6, 8] , we found impaired reliability for SED (min/day) compared to PA.","Consistent with previous studies START_CITE [ 6 , END_CITE CITE_b5 START_CITE 8 ] END_CITE CITE_b7 , we found impaired reliability for sedentary behavior ( min / day ) compared to physical activity .","Consistent with [prior_study_ENTITY] START_CITE [ 6 , END_CITE CITE_b5 START_CITE 8 ] END_CITE CITE_b7 , we found impaired reliability for sedentary behavior ( min / day ) compared to physical activity .",11,Discussion,NaN,PARAGRAPH,"['1_b5', '1_b7']",2,52129671
213,1,1_4122_4187,1_31,"While several studies have found that 2-6 days are required [5, 7, 8, 6, 30] , other studies have shown that 12 [4] and 16-23 days are needed [3] .","While several studies have found that 2 - 6 days are required START_CITE [ 5 , END_CITE CITE_b4 START_CITE 7 , END_CITE CITE_b6 START_CITE 8 , END_CITE CITE_b7 START_CITE 6 , END_CITE CITE_b5 START_CITE 30 ] END_CITE CITE_b29 , other studies have shown that 12 ST

In [29]:
df.loc[df.reference_key == '1_b5'].citation_context

1783    However, estimates of how many days of monitoring that should be included to obtain a reliable result vary considerably between studies [3-7, 2, 8], and might also vary between outcome variables of interest [#CITATION_TAG, 8].
Name: citation_context, dtype: object

In [31]:
from collections import Counter
import pandas as pd
import ast

def count_common_words(sentence1, sentence2):
    words1 = set(sentence1.split())
    words2 = set(sentence2.split())
    return len(words1 & words2)

def find_most_similar_sentence(row, sent_df):
    reference_key = row['reference_key']
    citation_context = row['citation_context']
    # Find sentences with the same reference_key
    same_key_df = sent_df[sent_df['references'].apply(lambda x: reference_key in ast.literal_eval(x))]
    # If no sentence has the same reference_key, return None
    if same_key_df.empty:
        return None
    # Calculate common words count for each sentence and find the sentence with the highest count
    same_key_df['common_words_count'] = same_key_df['sentence_original'].apply(lambda x: count_common_words(citation_context, x))
    most_similar_sentence = same_key_df.loc[same_key_df['common_words_count'].idxmax(), 'sentence_original']
    return most_similar_sentence

# Apply the function to df
df['most_similar_sentence'] = df.apply(lambda x: find_most_similar_sentence(x, sent), axis=1)


In [35]:
df[['citation_context', 'most_similar_sentence']]

,citation_context,most_similar_sentence
0,"In historical research, regressive methods moving step-by-step back in time from a better known later situation were already applied by #CITATION_TAG or by Bloch (1931) for the reconstruction of medieval agrarian landscapes in France and have been used since then within historical research (e.g. Forschungsinitiative Umweltgeschichte 1999).",None
1,"For example, a new cut-off channel at the Donaukanal excavated in 1700-1703 and bridges built in 1704 were later added, so it could serve as the basis for proposed hydraulic constructions in 1712 (#CITATION_TAG).","For example, a new cut-off channel at the Donaukanal excavated in 1700-1703 and bridges built in 1704 were later added, so it could serve as the basis for proposed hydraulic constructions in 1712 (Slezak 1977) ."
2,"It can be considered as the first map of the Viennese riverscape, which is depicted in a geographically largely correct manner (Mohilla and Michlmayr 1996; #CITATION_TAG).","It can be considered as the first map of the Viennese riverscape, which is depicted in a geographically largely correct manner (Mohilla and Michlmayr 1996; Opll 2004 )."
3,"According to the ''hydraulic geometry'' approach introduced by #CITATION_TAG, channel forms respond to changes in the flow regime.","According to the ''hydraulic geometry'' approach introduced by Leopold and Maddock (1953) , channel forms respond to changes in the flow regime."
4,"Historians ask for the motivation and interests that were important in the making, using and keeping of their sources (#CITATION_TAG).",None
...,...,...
3995,"In order to conclude whether identified fluvial dynamics reflect the river's typical behaviour rather than an exceptional hydromorphological state, climatic changes and related flood regimes also have to be considered (#CITATION_TAG; McCarney-Castle et al. 2011; Macklin et al. 2012).","In order to conclude whether identified fluvial dynamics reflect the river's typical behaviour rather than an exceptional hydromorphological state, climatic changes and related flood regimes also have to be considered (Howard 1996; McCarney-Castle et al. 2011; Macklin et al. 2012 channel straightening and profile widening."
3996,"It can be considered as the first map of the Viennese riverscape, which is depicted in a geographically largely correct manner (#CITATION_TAG; Opll 2004).","It can be considered as the first map of the Viennese riverscape, which is depicted in a geographically largely correct manner (Mohilla and Michlmayr 1996; Opll 2004 )."
3997,"Borders of land properties, hunting grounds or administrative borders also proved to be very useful; in particular, the so-called Burgfriedsgrenze, the jurisdiction border of the Viennese magistrate established in the Late Middle Ages that stretched far north into the floodplain (#CITATION_TAG).","Borders of land properties, hunting grounds or administrative borders also proved to be very useful; in particular, the so-called Burgfriedsgrenze, the jurisdiction border of the Viennese magistrate established in the Late Middle Ages that stretched far north into the floodplain (Opll 1986) ."
3998,"Due to the high bed shear stress, new channels incised into the floodplain terrain (first order avulsion) or led to the reoccupation of abandoned arms or crevasse channels (second order avulsion; #CITATION_TAG).","Due to the high bed shear stress, new channels incised into the floodplain terrain (first order avulsion) or led to the reoccupation of abandoned arms or crevasse channels (second order avulsion; Richards et al. 1993) ."


In [15]:
df.duplicated().sum()

0

In [5]:
df = pd.read_csv('newest_test.csv')

In [12]:
df[['citation_context','reference_key', 'most_similar_sentence']].head(60)

,citation_context,reference_key,most_similar_sentence
0,"In historical research, regressive methods moving step-by-step back in time from a better known later situation were already applied by #CITATION_TAG or by Bloch (1931) for the reconstruction of medieval agrarian landscapes in France and have been used since then within historical research (e.g. Forschungsinitiative Umweltgeschichte 1999).",NaN,NaN
1,"For example, a new cut-off channel at the Donaukanal excavated in 1700-1703 and bridges built in 1704 were later added, so it could serve as the basis for proposed hydraulic constructions in 1712 (#CITATION_TAG).",96_b60,"For example, a new cut-off channel at the Donaukanal excavated in 1700-1703 and bridges built in 1704 were later added, so it could serve as the basis for proposed hydraulic constructions in 1712 (Slezak 1977) ."
2,"It can be considered as the first map of the Viennese riverscape, which is depicted in a geographically largely correct manner (Mohilla and Michlmayr 1996; #CITATION_TAG).",96_b48,"It can be considered as the first map of the Viennese riverscape, which is depicted in a geographically largely correct manner (Mohilla and Michlmayr 1996; Opll 2004 )."
3,"According to the ''hydraulic geometry'' approach introduced by #CITATION_TAG, channel forms respond to changes in the flow regime.",96_b37,"According to the ''hydraulic geometry'' approach introduced by Leopold and Maddock (1953) , channel forms respond to changes in the flow regime."
4,"Historians ask for the motivation and interests that were important in the making, using and keeping of their sources (#CITATION_TAG).",96_b10,NaN
5,"In historical research, regressive methods moving step-by-step back in time from a better known later situation were already applied by Seebohm (1883) or by #CITATION_TAG for the reconstruction of medieval agrarian landscapes in France and have been used since then within historical research (e.g. Forschungsinitiative Umweltgeschichte 1999).",NaN,NaN
6,"These include, amongst others, the Baltimore-Chesapeake region (#CITATION_TAG), the American Great Plains (Cunfer 2008), the Rocky Mountains (Aspinall 2004), Southern Germany (Bender et al. 2005; Schuppert and Dix 2009), the Tisza River in Hungary (Hegedüs and Duray 2009) and several Austrian villages and rivers (Forschungsinitiative Umweltgeschichte 1999; Haidvogl 2008).",96_b15,"These include, amongst others, the Baltimore-Chesapeake region (Foresman et al. 1997) , the American Great Plains (Cunfer 2008) , the Rocky Mountains (Aspinall 2004) , Southern Germany (Bender et al. 2005; Schuppert and Dix 2009) , the Tisza River in Hungary (Hegedüs and Duray 2009) and several Austrian villages and rivers (Forschungsinitiative Umweltgeschichte 1999; Haidvogl 2008) . Knowles (2002) has already demonstrated the potential of GIS techniques for historical research."
7,"These include, amongst others, the Baltimore-Chesapeake region (Foresman et al. 1997), the American Great Plains (Cunfer 2008), the Rocky Mountains (Aspinall 2004), Southern Germany (Bender et al. 2005; Schuppert and Dix 2009), the Tisza River in Hungary (Hegedüs and Duray 2009) and several Austrian villages and rivers (Forschungsinitiative #CITATION_TAG; Haidvogl 2008).",96_b56,NaN
8,"In recent years, GISbased studies of historical fluvial morphology were conducted over the long term to reveal the causes of past channel changes and floodplain degradation (#CITATION_TAG (Gurnell et al., 2005 Marston et al. 1995; Kiss et al. 2008).",96_b20,NaN
9,"For example, we identified a long, vegetated ditch with some smaller backwaters in 1849 and 1817 that later proved to be the last remnant of the former Fugbach side arm in 1570 (compare #CITATION_TAG Hohensinner et al. 2013, in this issue).",96_b23,"For example, we identified a long, vegetated ditch with some smaller backwaters in 1849 and 1817 that later proved to be the last remnant of the former Fugbach side arm in 1570 (compare Haidvogl et al. 2013 and Hohensinner et 

In [22]:
def check_first_word(row):
    citation_first_word = row['citation_context'].split()[0].lower()
    sentence = row['most_similar_sentence']
    if pd.isna(sentence):  # Check if sentence is NaN
        return True
    sentence_first_word = sentence.split()[0].lower()
    return citation_first_word != sentence_first_word

df_non_matching = df[df.apply(check_first_word, axis=1)]

In [25]:
df_non_matching[['citation_context','reference_key', 'most_similar_sentence']].head(60)

,citation_context,reference_key,most_similar_sentence
0,"In historical research, regressive methods moving step-by-step back in time from a better known later situation were already applied by #CITATION_TAG or by Bloch (1931) for the reconstruction of medieval agrarian landscapes in France and have been used since then within historical research (e.g. Forschungsinitiative Umweltgeschichte 1999).",NaN,NaN
4,"Historians ask for the motivation and interests that were important in the making, using and keeping of their sources (#CITATION_TAG).",96_b10,NaN
5,"In historical research, regressive methods moving step-by-step back in time from a better known later situation were already applied by Seebohm (1883) or by #CITATION_TAG for the reconstruction of medieval agrarian landscapes in France and have been used since then within historical research (e.g. Forschungsinitiative Umweltgeschichte 1999).",NaN,NaN
7,"These include, amongst others, the Baltimore-Chesapeake region (Foresman et al. 1997), the American Great Plains (Cunfer 2008), the Rocky Mountains (Aspinall 2004), Southern Germany (Bender et al. 2005; Schuppert and Dix 2009), the Tisza River in Hungary (Hegedüs and Duray 2009) and several Austrian villages and rivers (Forschungsinitiative #CITATION_TAG; Haidvogl 2008).",96_b56,NaN
8,"In recent years, GISbased studies of historical fluvial morphology were conducted over the long term to reveal the causes of past channel changes and floodplain degradation (#CITATION_TAG (Gurnell et al., 2005 Marston et al. 1995; Kiss et al. 2008).",96_b20,NaN
10,"The major channel shifts at the Viennese Danube therefore have to be interpreted against the background of the Grindelwald Fluctuation, the first extreme phase of the Little Ice Age from the 1560s to the 1620s (Pfister 1980 (Pfister, 2007 #CITATION_TAG; Hohensinner et al. 2013, in this issue).",NaN,NaN
12,"Annual erosion rates ranged from 1.6 % of the floodplain terrain in the Lobau directly downstream from Vienna to 2.5 % in the more dynamic Danube sections such as the Machland, 160 km upstream from Vienna (#CITATION_TAG).",96_b1,"However, the comparison with the famous ''Jagdatlas Kaiser Karls VI.'' (''Atlas of imperial hunting grounds'') produced by J. J. Marinoni between 1726 and 1729 reveals that the riverscape had experienced substantial alterations since 1704. 13 This map series is the first geometrically coherent cartographic source that also covers areas remote from the historical city centre (Marinoni 1751) . 14 In the 18th century, the growth of the city also gave rise to new regulation projects."
13,#CITATION_TAG he Bland-Altman method provides insight into the distribution of differences in relation to mean values.,109_b24,"Results between these ranges represent moderate-to-good reliability. 27 According to another reference, ICC >0.7 is considered good. 25 The Bland-Altman method provides insight into the distribution of differences in relation to mean values. 28 Agreement was quantified by calculating the mean difference between two sets of observations and the SD for this difference."
14,"#CITATION_TAG owever, SDD ranged from 4 to 6, indicating less precision of TP examination than reliability.",109_b29,"The median TP count of 8 was elevated as compared with the median TP count in the general population, which is between 3 and 6 TPs. 6 Previously, it has been shown that TP counts were elevated in regional pain conditions as compared with pain-free controls, but lower than in fibromyalgia. 30 However, SDD ranged from 4 to 6, indicating less precision of TP examination than reliability."
20,"One simple but effective strategy is sequential search in which the animal keeps searching until it finds an option that exceeds a threshold of acceptability [2, #CITATION_TAG].",180_b13,"To understand how a collective decision is reached, the rules followed by contributing members of the group must be identified [14] ."


In [27]:
test = pd.read_csv('df_with_sent_id.csv')

In [29]:
test.most_similar_sentence_id.isna().sum()

611

In [30]:
df.most_similar_sentence.isna().sum()

611

In [3]:
tfid = pd.read_csv('df_Tfidf.csv')

In [8]:
tfid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_id                 4000 non-null   object 
 1   core_id                   4000 non-null   int64  
 2   citation_offset           3995 non-null   float64
 3   total_doc_length          4000 non-null   int64  
 4   section_info              4000 non-null   object 
 5   citing_title              4000 non-null   object 
 6   citing_author             4000 non-null   object 
 7   citing_publication_info   4000 non-null   int64  
 8   citing_abstract           3992 non-null   object 
 9   cited_title               4000 non-null   object 
 10  cited_author              4000 non-null   object 
 11  cited_abstract            3040 non-null   object 
 12  cited_doi                 3646 non-null   object 
 13  cited_publication_date    3988 non-null   float64
 14  cited_pu

In [22]:
test = tfid[['unique_id','reference_key', 'citation_context',  'most_similar_sentence', 'similarity_score', 'most_similar_sentence_id']]#.head(60)

In [24]:
test#.loc[(test.similarity_score.isna() == True) & (test.reference_key.isna() == True)]

,unique_id,reference_key,citation_context,most_similar_sentence,similarity_score,most_similar_sentence_id
0,CC1,NaN,"In historical research, regressive methods mov...",NaN,NaN,NaN
1,CC2,96_b60,"For example, a new cut-off channel at the Dona...","For example, a new cut-off channel at the Dona...",0.937115,96_3229_3276
2,CC3,96_b48,It can be considered as the first map of the V...,It can be considered as the first map of the V...,0.898220,96_3080_3118
3,CC4,96_b37,According to the ''hydraulic geometry'' approa...,According to the ''hydraulic geometry'' approa...,0.814947,96_6945_6977
4,CC5,96_b10,Historians ask for the motivation and interest...,NaN,NaN,NaN
...,...,...,...,...,...,...
3995,CC3996,96_b32,In order to conclude whether identified fluvia...,In order to conclude whether identified fluvia...,0.867528,96_2117_2181
3996,CC3997,96_b45,It can be considered as the first map of the V...,It can be considered as the first map of the V...,0.833732,96_3080_3118
3997,CC3998,96_b47,"Borders of land properties, hunting grounds or...","Borders of land properties, hunting grounds or...",0.957745,96_7102_7156
3998,CC3999,96_b54,"Due to the high bed shear stress, new channels...","Due to the high bed shear stress, new channels...",0.906395,96_5283_5329


In [25]:
test.to_excel('annotate_me.xlsx', index=False)

In [13]:
pip install openpyxl

     |████████████████████████████████| 249 kB 3.2 MB/s eta 0:00:01
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


# frequency based

## density

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_id                 4000 non-null   object 
 1   core_id                   4000 non-null   int64  
 2   citation_offset           3995 non-null   float64
 3   total_doc_length          4000 non-null   int64  
 4   section_info              4000 non-null   object 
 5   citing_title              4000 non-null   object 
 6   citing_author             4000 non-null   object 
 7   citing_publication_info   4000 non-null   int64  
 8   citing_abstract           3992 non-null   object 
 9   cited_title               4000 non-null   object 
 10  cited_author              4000 non-null   object 
 11  cited_abstract            3040 non-null   object 
 12  cited_doi                 3646 non-null   object 
 13  cited_publication_date    3988 non-null   float64
 14  cited_pu

# popularity

In [10]:
# Calculate article length
article_length = cite.groupby('article_id')['sentence_end'].max().reset_index()
article_length.columns = ['article_id', 'article_length']

In [12]:
# Count sentences per article
sentence_count = cite.groupby('article_id')['sentence_id'].count().reset_index()
sentence_count.columns = ['article_id', 'sentence_count']

In [13]:
# Merge the two dataframes together
article_info = pd.merge(article_length, sentence_count, on='article_id')


In [15]:
# Calculate average density of citations per article
article_info['citation_density'] = article_info['sentence_count'] / article_info['article_length']

In [18]:
article_info.citation_density.min()

0.001151576219951058

In [19]:
# Calculate section length and sentence count
section_length = cite.groupby(['article_id', 'section_nr'])['sentence_end'].max().reset_index()
section_length.columns = ['article_id', 'section_nr', 'section_length']

sentence_count = cite.groupby(['article_id', 'section_nr'])['sentence_id'].count().reset_index()
sentence_count.columns = ['article_id', 'section_nr', 'sentence_count']

# Merge the two dataframes together
section_info = pd.merge(section_length, sentence_count, on=['article_id', 'section_nr'])

# Calculate average density of citations per section
section_info['citation_density'] = section_info['sentence_count'] / section_info['section_length']


In [23]:
section_info

,article_id,section_nr,section_length,sentence_count,citation_density
0,1,1,1416,30,0.021186
1,1,3,1920,4,0.002083
2,1,4,2590,7,0.002703
3,1,11,5212,45,0.008634
4,1,12,5335,3,0.000562
...,...,...,...,...,...
1518,229,2,2371,3,0.001265
1519,229,4,2471,1,0.000405
1520,229,5,3014,3,0.000995
1521,229,6,3986,9,0.002258


In [24]:
from sklearn.preprocessing import MinMaxScaler

# initialize the scaler
scaler = MinMaxScaler()

# perform the scaling on citation_density column and replace the original data
# the scaler expects a 2D array or DataFrame, so we use double square brackets [[]] to give it one
article_info['citation_density'] = scaler.fit_transform(article_info[['citation_density']])

# if you want to scale the 'sentence_count' as well
article_info['sentence_count'] = scaler.fit_transform(article_info[['sentence_count']])


In [30]:
article_info

,article_id,article_length,sentence_count,citation_density
0,1,5335,0.132588,0.531736
1,2,10177,0.150160,0.296995
2,3,5872,0.047923,0.170477
3,4,9246,0.113419,0.245702
4,5,7588,0.135783,0.371173
...,...,...,...,...
223,225,9408,0.055911,0.109780
224,226,6206,0.068690,0.230899
225,227,4701,0.059105,0.273745
226,228,3334,0.065495,0.443228


In [32]:
section_info

,article_id,section_nr,section_length,sentence_count,citation_density
0,1,1,1416,30,0.021186
1,1,3,1920,4,0.002083
2,1,4,2590,7,0.002703
3,1,11,5212,45,0.008634
4,1,12,5335,3,0.000562
...,...,...,...,...,...
1518,229,2,2371,3,0.001265
1519,229,4,2471,1,0.000405
1520,229,5,3014,3,0.000995
1521,229,6,3986,9,0.002258


In [35]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Calculate article length
article_length = cite.groupby('article_id')['sentence_end'].max().reset_index()
article_length.columns = ['article_id', 'article_length']

# Count sentences per article
sentence_count = cite.groupby('article_id')['sentence_id'].count().reset_index()
sentence_count.columns = ['article_id', 'sentence_count']

# Merge the two dataframes together
article_info = pd.merge(article_length, sentence_count, on='article_id')

# Calculate citation density
article_info['citation_density'] = article_info['sentence_count'] / article_info['article_length']

# Apply MinMaxScaler to citation_density
article_info['citation_density'] = scaler.fit_transform(article_info[['citation_density']])

# Merge citation_density into original dataframe
cite = pd.merge(cite, article_info[['article_id', 'citation_density']], on='article_id', how='left')

# Calculate section length and sentence count
section_length = cite.groupby(['article_id', 'section_nr'])['sentence_end'].max().reset_index()
section_length.columns = ['article_id', 'section_nr', 'section_length']

sentence_count = cite.groupby(['article_id', 'section_nr'])['sentence_id'].count().reset_index()
sentence_count.columns = ['article_id', 'section_nr', 'sentence_count']

# Merge the two dataframes together
section_info = pd.merge(section_length, sentence_count, on=['article_id', 'section_nr'])

# Calculate section density
section_info['section_density'] = section_info['sentence_count'] / section_info['section_length']

# Apply MinMaxScaler to section_density
section_info['section_density'] = scaler.fit_transform(section_info[['section_density']])

# Merge section_density into original dataframe
cite = pd.merge(cite, section_info[['article_id', 'section_nr', 'section_density']], on=['article_id', 'section_nr'], how='left')


In [46]:
cite

,article_id,sentence_id,para_id,section_nr,section_title,subsection_title,reference_key,reference_citekey,journal_latest,paper_citekey,...,sentence_start_zero,sentence_end_zero,sentence_middle_zero,section_start_new,section_end_new,section_length_new,citation_density_x,citation_density_y,citation_density,section_density
0,1,1_456_509,1_3,1,Introduction,NaN,1_b0,hutcheon_j_random_measurement_error_and_regres...,bmj,aadland_e_reliability_of_objectively_measured_...,...,0,53,26.5,456,1416,960,0.531736,0.531736,0.531736,0.387564
1,1,1_515_540,1_4,1,Introduction,NaN,1_b1,trost_s_conducting_accelerometerbased_activity...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,59,84,71.5,456,1416,960,0.531736,0.531736,0.531736,0.387564
2,1,1_540_593,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,84,137,110.5,456,1416,960,0.531736,0.531736,0.531736,0.387564
3,1,1_540_593,1_4,1,Introduction,NaN,1_b7,hart_t_how_many_days_of_monitoring_predict_phy...,int journal of behav nutr phys act,aadland_e_reliability_of_objectively_measured_...,...,84,137,110.5,456,1416,960,0.531736,0.531736,0.531736,0.387564
4,1,1_593_625,1_4,1,Introduction,NaN,1_b5,matthews_c_sources_of_variance_in_daily_physic...,med science sports exerc,aadland_e_reliability_of_objectively_measured_...,...,137,169,153.0,456,1416,960,0.531736,0.531736,0.531736,0.387564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19421,229,229_3998_4037,229_39,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",...,0,39,19.5,3998,4878,880,0.318532,0.318532,0.318532,0.018000
19422,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b28,knowlton_b_the_information_acquired_during_art...,journal of experimental psychology learning me...,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",...,462,553,507.5,3998,4878,880,0.318532,0.318532,0.318532,0.018000
19423,229,229_4460_4551,229_50,8,Methods ( extended ),NaN,229_b37,redington_m_transfer_in_artificial_grammar_lea...,journal of experimental psychology general,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",...,462,553,507.5,3998,4878,880,0.318532,0.318532,0.318532,0.018000
19424,229,229_4739_4768,229_50,8,Methods ( extended ),NaN,229_b44,zimmerer_c_2014_employed_the_same_randomized_s...,book,"zimmerer_v_a_case_of_""order_insensitivity""_nat...",...,741,770,755.5,3998,4878,880,0.318532,0.318532,0.318532,0.018000


In [48]:
pd.merge(left = df, right = cite[['reference_key', 'citation_density']], on = 'reference_key', how='left')

,unique_id,core_id,citation_offset,total_doc_length,section_info,citing_title,citing_author,citing_publication_info,citing_abstract,cited_title,...,co_mentions,citation_class_label,citation_influence_label,cite_pos_in_sent,sent_pos_in_article,article_id,citing_publication_dat,reference_key,diff_publication_date,citation_density
0,CC1,81153632,2302.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,The English Village Community Examined in its ...,...,1.0,5,0,0.398827,0.038544,96,2013,NaN,NaN,NaN
1,CC2,81153632,23562.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,Die italienischen Begründer der Wiener Donauka...,...,1.0,0,1,0.929245,0.394515,96,2013,96_b60,36.0,0.202116
2,CC3,81153632,22736.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,Wien im Bild historischer Karten Die Entwicklu...,...,2.0,4,1,0.912281,0.380684,96,2013,96_b48,9.0,0.202116
3,CC4,81153632,46410.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,The hydraulic geometry of stream channels and ...,...,0.0,0,0,0.484615,0.777075,96,2013,96_b37,60.0,0.202116
4,CC5,81153632,4191.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,From memory to written record,...,0.0,0,0,0.888060,0.070173,96,2013,96_b10,20.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7101,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.320598
7102,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.320598
7103,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.320598
7104,CC4000,187054721,2875.0,26616,"{""Research Article"": 0, ""Towards Sustainable S...",Towards Sustainable Self-Compacting Concrete: ...,Hisham Qasrawi,2018,Steel industry results in accumulation of stee...,Use of Relatively High Fe2O3 Steel Slag as Coa...,...,2.0,2,1,0.811475,0.108018,175,2018,175_b0,6.0,0.320598


# enriching by year

## publication year of citing paper (some from core, some manual)

In [10]:
data.citing_title[200]

'Knee Arthroscopy Cohort Southern Denmark (KACS): protocol for a prospective cohort study'

In [7]:
data.core_id[200]

NameError: name 'data' is not defined

In [12]:
data.core_id.value_counts()

93144659     120
81800077      93
82852800      86
81068384      74
52040220      58
            ... 
82260752       1
71826501       1
186895161      1
82173079       1
16246214       1
Name: core_id, Length: 229, dtype: int64

In [12]:
## calling the api

In [4]:
data.citation_context[20]

'One simple but effective strategy is sequential search in which the animal keeps searching until it finds an option that exceeds a threshold of acceptability [2, #CITATION_TAG].'

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 21 columns):
unique_id                   4000 non-null object
core_id                     4000 non-null int64
citation_offset             3995 non-null float64
total_doc_length            4000 non-null int64
section_info                4000 non-null object
citing_title                4000 non-null object
citing_author               4000 non-null object
citing_publication_info     3705 non-null object
citing_abstract             3992 non-null object
cited_title                 4000 non-null object
cited_author                4000 non-null object
cited_abstract              3040 non-null object
cited_doi                   3646 non-null object
cited_publication_date      3888 non-null object
cited_publication_info      3928 non-null object
citation_context            4000 non-null object
self_citation               3578 non-null float64
direct_citations            3932 non-null float64
co_mention

In [6]:
# fetching publication data

In [4]:
api_key = ""
with open ("apikey.txt", "r") as apikey_file:
    api_key=apikey_file.readlines()[0].strip()
api_endpoint = "https://api.core.ac.uk/v3/"

In [5]:
def get_entity(url_fragment):
    headers={"Authorization":"Bearer "+api_key}
    response = requests.get(api_endpoint + url_fragment, headers=headers)
    if response.status_code == 200:
        return response.json(), response.elapsed.total_seconds()
    else:
        print(f"Error code {response.status_code}, {response.content}")

In [6]:
def get_date(core_id):
    try:
        data, elapsed = get_entity("outputs/" + core_id)
        date = data['publishedDate']
        return date
    except:
        return None

In [4]:
data.core_id.value_counts()

NameError: name 'data' is not defined

In [12]:
get_date("52040220")

Error code 404, b'{"message":""}'


In [ ]:
data['citing_publication_date'] = data.cor

In [ ]:
data.core_id.apply(lambda x: get_date(str(x)))

Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'


In [3]:
unique_core_id = data['core_id'].unique()

In [7]:
unique_core_id = pd.DataFrame(unique_core_id, columns=['column_name'])

In [8]:
unique_core_id['citing_publication_date'] = unique_core_id.column_name.apply(lambda x: get_date(str(x)))

Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 410, b'"The output you were looking for is disabled."'
Error code 410, b'"The output you were looking for is disabled."'
Error code 410, b'"The output you were looking for is disabled."'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 404, b'{"message":""}'
Error code 410, b'"The output you were looking for is disabled."'
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, 

In [62]:
unique_core_id['year_column'] = unique_core_id['citing_publication_date'].str.extract(r'(\d{4})', expand=False)

In [10]:
unique_core_id.year_column.isna().sum()

172

In [11]:
unique_core_id.citing_publication_date.isna().sum()

131

In [63]:
unique_core_id.year_column.value_counts()

2015    14
2014    11
2012    10
2013     7
2011     5
2016     3
2018     3
1998     1
2010     1
2017     1
2019     1
Name: year_column, dtype: int64

In [6]:
unique_core_id = pd.read_csv('data/edited_unique_core_ids.csv', sep=';')

In [11]:
unique_core_id.year_column.isna().sum()

0

In [7]:
df['citing_publication_dat'] = pd.merge(left=df, right= unique_core_id[['core_id', 'year_column']], on = 'core_id', how = 'left').year_column#.year_column.value_counts()#.isna().sum() / len(data)

In [8]:
test = df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_id                 4000 non-null   object 
 1   core_id                   4000 non-null   int64  
 2   citation_offset           3995 non-null   float64
 3   total_doc_length          4000 non-null   int64  
 4   section_info              4000 non-null   object 
 5   citing_title              4000 non-null   object 
 6   citing_author             4000 non-null   object 
 7   citing_publication_info   4000 non-null   int64  
 8   citing_abstract           3992 non-null   object 
 9   cited_title               4000 non-null   object 
 10  cited_author              4000 non-null   object 
 11  cited_abstract            3040 non-null   object 
 12  cited_doi                 3646 non-null   object 
 13  cited_publication_date    3888 non-null   object 
 14  cited_pu

In [18]:
unique_core_id.citing_publication_date.isna().sum()

131

In [32]:
unique_core_id

,column_name,citing_publication_date,year_column,core_id
0,81153632,None,NaN,81153632
1,50694047,None,NaN,50694047
2,5225915,2011-05-24T00:00:00+01:00,2011,5225915
3,82875399,2014-01-01T00:00:00+00:00,2014,82875399
4,82218905,,NaN,82218905
...,...,...,...,...
224,52040210,None,None,52040210
225,82867877,None,None,82867877
226,1385821,None,None,1385821
227,82867364,None,None,82867364


In [30]:
year = pd.merge(left=unique_core_id, right=data[['core_id', 'citing_title']], on='core_id', how = 'left')

In [36]:
year = year.drop_duplicates('column_name')

In [37]:
year.to_csv('unique_core_id_with_year.csv')

In [20]:
test = unique_core_id.loc[unique_core_id.citing_publication_date.isna() == True]

In [21]:
test

,column_name,citing_publication_date,year_column,core_id
0,81153632,None,None,81153632
1,50694047,None,None,50694047
8,81848562,None,None,81848562
9,397819,None,None,397819
19,5255546,None,None,5255546
...,...,...,...,...
224,52040210,None,None,52040210
225,82867877,None,None,82867877
226,1385821,None,None,1385821
227,82867364,None,None,82867364


In [22]:
test['citing_publication_date'] = test.column_name.apply(lambda x: get_date(str(x)))

Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''
Error code 429, b''


<ipython-input-22-987976d08a1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['citing_publication_date'] = test.column_name.apply(lambda x: get_date(str(x)))


In [24]:
test.citing_publication_date.isna().sum()

131

In [42]:
data.loc[data.citing_title.str.contains('Iss. 1 , Article 3') == True]#.citing_abstract

,unique_id,core_id,citation_offset,total_doc_length,section_info,citing_title,citing_author,citing_publication_info,citing_abstract,cited_title,...,cited_abstract,cited_doi,cited_publication_date,cited_publication_info,citation_context,self_citation,direct_citations,co_mentions,citation_class_label,citation_influence_label
959,CC960,186934554,5735.0,35137,"{""1 Introduction"": 1252, ""2 Description of the...","Iss. 1 , Article 3",Rand Wilcox,NaN,"For two independent groups, let M j (x) be som...",Pairwise Multiple Comparisons in the Unequal V...,...,Abstract Four pairwise multiple comparison pro...,10.1080/01621459.1980.10477552,1980-12,Journal of the American Statistical Association,The method is based in part on Yuen's (1974) m...,NaN,1.0,0.0,1,0


In [18]:
edited_data = pd.read_csv('/home/roland/Projects/JP_citation_classification/feature_scraping/data/edited_unique_core_ids.csv', sep=';')

In [23]:
edited_data = edited_data.drop(columns=['Column1'])

In [25]:
edited_data = edited_data.drop(columns=['column_name'])

In [27]:
edited_data['citing_year'] = edited_data.year_column

In [28]:
edited_data

,citing_publication_date,year_column,core_id,citing_title,citing_year
0,1998-12-01T00:00:00+00:00,1998,82899510,A matter of opinion,1998
1,2010-01-19T00:00:00+00:00,2010,76995576,Evaluating diagnostic tests: the area under th...,2010
2,2011-05-24T00:00:00+01:00,2011,5225915,A Simple Threshold Rule Is Sufficient to Expla...,2011
3,2011-01-01T00:00:00+00:00,2011,35467497,What is dust?-Physical foundations of the aver...,2011
4,2011-01-01T00:00:00+00:00,2011,82878092,Carbohydrate-Biased Control of Energy Metaboli...,2011
...,...,...,...,...,...
224,NaN,2012,52040210,Governance costs in foreign direct investments...,2012
225,NaN,2013,82867877,Pathology and pathogenesis of vascular cogniti...,2013
226,NaN,2011,1385821,Policy and practice in the use of root cause a...,2011
227,NaN,2013,82867364,Wandering tales: evolutionary origins of menta...,2013


In [34]:
data = data.drop(columns=['citing_title'])

In [36]:
data = pd.merge(left=data, right = edited_data[['core_id', 'citing_year', 'citing_title']], on = 'core_id', how = 'left')

In [38]:
data.citing_year.value_counts()

2015    994
2014    913
2013    658
2012    528
2011    222
2009    207
2016    124
2010    107
2018     74
2019     60
2008     54
2017     44
2005      6
2006      4
1998      3
2001      2
Name: citing_year, dtype: int64

## publication year of cited paper

In [97]:
df.loc[(df.article_id == 2) & (df.cited_publication_date.isna() == True)]#.cited_title#.str.lower().isin(ref.loc[ref.article_id == 1].title.str.lower())

,unique_id,core_id,citation_offset,total_doc_length,section_info,citing_author,citing_publication_info,citing_abstract,cited_title,cited_author,...,self_citation,direct_citations,co_mentions,citation_class_label,citation_influence_label,citing_year,citing_title,cite_pos_in_sent,sent_pos_in_article,article_id
3739,CC3740,52040220,26998.0,60848,"{""1. Introduction"": 2057, ""2. Business network...",Morten Abrahamsen,Industrial Marketing Management,The issue of how different actors in a network...,Using Templates in the Thematic Analysis of Text,"[""Nigel King""]",...,0.0,2.0,1.0,5,1,2012,Sensemaking in business networks: Introducing ...,0.746544,0.443696,2
3770,CC3771,52040220,6874.0,60848,"{""1. Introduction"": 2057, ""2. Business network...",Morten Abrahamsen,Industrial Marketing Management,The issue of how different actors in a network...,Networks in Socioeconomic Systems: A Critical ...,"[""Luis Araujo"", ""Geoffrey Easton""]",...,0.0,1.0,2.0,0,0,2012,Sensemaking in business networks: Introducing ...,0.286164,0.112970,2


In [8]:
ref = pd.read_csv('data/causeminer_data/references_merged.csv')

In [101]:
ref.loc[ref.article_id == 2].title.str.lower().sort_values().tail(60)

10558    developing network pictures as a research tool...
5955         developing relationships in business networks
12202    distribution channels as political economies a...
13939    does trust matter exploring the effects of int...
339      dyadic business relationships within a busines...
4917     exploring strategy perceptions in changing env...
5131     from dyadic change to changing business networ...
9495     from value chain to value constellation design...
5479     going beyond customers a business segmentation...
10169    good case research in industrial marketing ins...
12193    henneberg is a professor of marketing and stra...
5956     how should companies interact in business netw...
13463    idea logics and network theory in business mar...
2897       impact of strategic alliances on firm valuation
3941     industry and strategic group dynamics competit...
12884    interaction relationships and networks in busi...
4105     making sense of network dynamics through netwo.

## enriching by time difference between citing and cited paper

In [152]:
df['diff_publication_date'] = df.citing_publication_dat - df.cited_publication_date

# Enriching by OpenAlex Data

In [5]:
oa = pd.read_csv('open_alex_enrichment.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_id                 4000 non-null   object 
 1   core_id                   4000 non-null   int64  
 2   citation_offset           3995 non-null   float64
 3   total_doc_length          4000 non-null   int64  
 4   section_info              4000 non-null   object 
 5   citing_title              4000 non-null   object 
 6   citing_author             4000 non-null   object 
 7   citing_publication_info   4000 non-null   int64  
 8   citing_abstract           3992 non-null   object 
 9   cited_title               4000 non-null   object 
 10  cited_author              4000 non-null   object 
 11  cited_doi                 3646 non-null   object 
 12  cited_publication_date    3988 non-null   float64
 13  cited_publication_info    3971 non-null   object 
 14  citation

In [10]:
oa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_id                 4000 non-null   object 
 1   core_id                   4000 non-null   int64  
 2   citation_offset           3995 non-null   float64
 3   total_doc_length          4000 non-null   int64  
 4   section_info              4000 non-null   object 
 5   citing_title              4000 non-null   object 
 6   citing_author             4000 non-null   object 
 7   citing_publication_info   3705 non-null   object 
 8   citing_abstract           3992 non-null   object 
 9   cited_title               4000 non-null   object 
 10  cited_author              4000 non-null   object 
 11  cited_doi                 3646 non-null   object 
 12  cited_publication_date    3888 non-null   object 
 13  cited_publication_info    3928 non-null   object 
 14  citation

In [12]:
df.drop(columns=['cited_abstract'], inplace=True)

In [16]:
df = pd.merge(left=df, right=oa[['unique_id', 'cited_abstract', 'cited_oa', 'cited_concepts', 'citing_oa', 'citing_concepts']], on= 'unique_id', how = 'left')

# saving the data

In [17]:
df.to_csv('data/enriched_data/7_added_causeminer_data.csv', index = False)

In [14]:
papers.to_csv('data/causeminer_data/papers_with_core_id.csv', index = False)

In [21]:
sent.to_csv('data/causeminer_data/sentences_with_core_id.csv', index = False)

In [27]:
ref.to_csv('data/causeminer_data/references_with_core_id.csv', index = False)

In [223]:
cite.to_csv('data/causeminer_data/citations_with_pos.csv', index = False)

In [36]:
df.to_csv('newest_test.csv', index = False)